In [1]:
# Built-in libraries
import logging
import os
import re
import subprocess
import time

# Data manipulation
import numpy as np
import pandas as pd

# Machine Learning
import optuna
from optuna.samplers import TPESampler
from sklearn.metrics import classification_report
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from sklearn.model_selection import TimeSeriesSplit

# Experiment tracking
import mlflow

# Visualization
from matplotlib import pyplot as plt

# Web scraping
import requests
from bs4 import BeautifulSoup
from unidecode import unidecode

# Utilities
from tqdm import tqdm

# DATA

## Obtaining match data from the website

In [2]:
class Match_data:
    def __init__(self, season, choice_1):
        """
        Initialize the class with the given URL and optional limits for the number of links, gameweeks, and scores.

        Args:
            url (str): The URL to scrape match data from.
            max (int or None): Maximum number of match links, gameweeks, and scores to scrape. If None, no limit is applied.
        """
        self.season = season
        self.url = None  # Store the URL as an attribute of the class
        self.max_links = None  # Limit for match links
        self.max_gameweeks = None  # Limit for gameweeks
        self.max_scores = None  # Limit for scores
        self.links = []  # List to store match links
        self.gameweeks = []  # List to store gameweek data
        self.scores = []  # List to store match scores (home and away goals)
        self.result = []  # List of results
        self.previous_gameweek = None  # Variable to store the last valid gameweek
        self.filename = None
        self.choice_1 = choice_1

    def get_links_gameweek_score(self):
        """
        Get match report links, gameweek data, result and match scores (home and away goals) from the provided URL.

        This function scrapes the provided URL for match report links, gameweek data, result and match scores,
        and stores them in the respective lists.

        Returns:
            list, list, list, list: Match report links, gameweek data, scores and results.
        """

        if os.path.exists(self.filename):
            df = pd.read_csv(self.filename)
            num_lines = len(df)
        else:
            num_lines = 0

        self.max_links = num_lines + self.choice_1
        self.max_gameweeks = num_lines + self.choice_1
        self.max_scores = num_lines + self.choice_1

        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:131.0) Gecko/20100101 Firefox/131.0",
            "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8",
            "Accept-Language": "en-US,en;q=0.5",
            "Accept-Encoding": "gzip, deflate, br",
            "Connection": "keep-alive",
            "Upgrade-Insecure-Requests": "1",
            "DNT": "1",
        }

        response = requests.get(self.url, headers=headers)
        soup = BeautifulSoup(response.text, "html.parser")

        self.links = []
        all_match_report_cells = soup.find_all('td', {'data-stat': 'match_report'})

        for td in all_match_report_cells:
            a = td.find('a')
            if a and a.get('href'):
                href = a['href']
                if href.startswith('/es/partidos/'):
                    full_url = f'https://fbref.com{href}'
                    self.links.append(full_url)

        self.links = self.links[num_lines:self.max_links]

        gameweek_cells = soup.find_all("th", {"data-stat": "gameweek"})
        self.gameweeks = []
        for cell in gameweek_cells:
            gameweek_value = cell.text.strip() if cell else "N/A"
            if gameweek_value in ["Sem.", ""]:
                continue
            if gameweek_value == "N/A" and self.previous_gameweek:
                gameweek_value = self.previous_gameweek
            try:
                gameweek_value = int(gameweek_value)
            except ValueError:
                gameweek_value = -1
            self.gameweeks.append(gameweek_value)
            self.previous_gameweek = gameweek_value

        self.gameweeks = self.gameweeks[num_lines:self.max_gameweeks]

        score_cells = soup.find_all("td", {"data-stat": "score"})
        self.scores = []
        self.result = []
        for cell in score_cells:
            score_text = cell.text.strip() if cell else ""
            if score_text:
                score_parts = score_text.split("–")
                if len(score_parts) == 2:
                    try:
                        home_goals = int(score_parts[0].strip())
                        away_goals = int(score_parts[1].strip())
                        self.scores.append((home_goals, away_goals))
                        if home_goals > away_goals:
                            result = 2
                        elif home_goals < away_goals:
                            result = 0
                        else:
                            result = 1
                        self.result.append(result)
                    except ValueError:
                        self.scores.append((0, 0))
                        self.result.append(0)

        self.scores = self.scores[num_lines:self.max_scores]
        self.result = self.result[num_lines:self.max_scores]

        return self.links, self.gameweeks, self.scores, self.result

    def get_match_data(self):
        """
        Create a CSV file with match data, combining gameweek, match information, and goals.

        This function combines match details (e.g., teams, date, time), gameweek data, and goals into a single DataFrame.

        Returns:
            pd.DataFrame: DataFrame containing match details, gameweek data, and goals.
        """

        # Create a DataFrame with empty values for the match data
        df = pd.DataFrame(
            {
                "id": pd.Series([None] * self.choice_1),
                "date_of_match": pd.Series([None] * self.choice_1),
                "hour_of_the_match": pd.Series([None] * self.choice_1),
                "day_of_week": pd.Series([None] * self.choice_1),
                "day_of_year": pd.Series([None] * self.choice_1),
                "hour_of_day": pd.Series([None] * self.choice_1),
                "home_team_name": pd.Series([None] * self.choice_1, dtype="str"),
                "away_team_name": pd.Series([None] * self.choice_1, dtype="str"),
                "home_trainer": pd.Series([None] * self.choice_1, dtype="str"),
                "away_trainer": pd.Series([None] * self.choice_1, dtype="str"),
                "stadium": pd.Series([None] * self.choice_1, dtype="str"),
                "attendance": pd.Series([None] * self.choice_1, dtype="str"),
                "capacity": pd.Series([None] * self.choice_1, dtype="Int64"),
                "attendance%": pd.Series([None] * self.choice_1, dtype="float"),
                "referee": pd.Series([None] * self.choice_1, dtype="str"),
                "var": pd.Series([None] * self.choice_1, dtype="str"),
                "home_team_lineup": pd.Series([None] * self.choice_1, dtype="str"),
                "away_team_lineup": pd.Series([None] * self.choice_1, dtype="str"),
                "home_possession": pd.Series([None] * self.choice_1, dtype="float"),
                "away_possession": pd.Series([None] * self.choice_1, dtype="float"),
            }
        )

        # Get the match data (links, gameweeks, and goals)
        links, gameweeks, scores, result = self.get_links_gameweek_score()

        # Create DataFrames from the gameweek, links, and scores data
        df_gameweeks = pd.DataFrame(gameweeks, columns=["gameweek"])
        df_links = pd.DataFrame(links, columns=["link"])
        df_scores = pd.DataFrame(scores, columns=["home_goals", "away_goals"])
        df_result = pd.DataFrame(result, columns=["result"])
        df_result["home_result"] = df_result["result"].apply(lambda x: 2 if x == 2 else (0 if x == 0 else 1))
        df_result["away_result"] = df_result["result"].apply(lambda x: 1 if x == 0 else (0 if x == 2 else 1))

        # Combine all DataFrames into one final DataFrame
        df_final = pd.concat([df_gameweeks, df, df_links, df_scores, df_result], axis=1)

        # Assign the final DataFrame to the class attribute with the new name
        self.df_final = df_final

    def get_statistics(self):
        """
        Get match statistics from the links and save them to a CSV file.

        This function extracts detailed statistics (e.g., team lineups, referee, attendance) from the match links.

        Returns:
            pd.DataFrame: DataFrame containing match statistics.
        """
        df = pd.read_csv(self.filename)
        id = df.iloc[-1]['id']

        for idx, link in enumerate(self.links):
            try:
                print(f"Processing link {idx}: {link}")
                headers = {
                    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:131.0) Gecko/20100101 Firefox/131.0",
                    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8",
                    "Accept-Language": "en-US,en;q=0.5",
                    "Accept-Encoding": "gzip, deflate, br",
                    "Connection": "keep-alive",
                    "Upgrade-Insecure-Requests": "1",
                    "DNT": "1",
                }
                response = requests.get(link, headers=headers)
                soup = BeautifulSoup(response.text, "html.parser")

                # Assign a unique ID for each match
                id += 1
                self.df_final.at[idx, "id"] = id

                # Extraer fecha y hora desde el HTML
                date_element = soup.find("span", {"class": "venuetime"})
                if date_element:
                    match_date = date_element.get("data-venue-date")  # ejemplo: "2025-03-02"
                    match_time = date_element.get("data-venue-time")  # ejemplo: "21:00"

                    # Asegurar formato limpio
                    match_date = pd.to_datetime(match_date).strftime('%Y-%m-%d')
                    match_time = pd.to_datetime(match_time, format='%H:%M', errors='coerce').strftime('%H:%M')

                    # Guardar directamente en columnas separadas
                    self.df_final.at[idx, "date_of_match"] = match_date
                    self.df_final.at[idx, "hour_of_the_match"] = match_time

                    # Opcional: también extraer día de la semana, día del año y hora decimal
                    full_datetime = pd.to_datetime(f"{match_date} {match_time}")
                    self.df_final.at[idx, "day_of_week"] = full_datetime.dayofweek
                    self.df_final.at[idx, "day_of_year"] = full_datetime.dayofyear
                    self.df_final.at[idx, "hour_of_day"] = full_datetime.hour + full_datetime.minute / 60

                # Extract team names
                teams_elements = soup.find_all("span", class_="teamandlogo")
                if len(teams_elements) >= 2:
                    home_team_name = teams_elements[0].text.strip()
                    away_team_name = teams_elements[1].text.strip()
                else:
                    print(f"Team names not found for link {link}. Assigning NaN.")
                    home_team_name = away_team_name = "NaN"
                # Remove accents and replace spaces with underscores
                home_team_name = unidecode(home_team_name).replace(" ", "_")
                away_team_name = unidecode(away_team_name).replace(" ", "_")
                # Assign the processed names to the DataFrame
                self.df_final.at[idx, "home_team_name"] = home_team_name
                self.df_final.at[idx, "away_team_name"] = away_team_name

                # Extract coaches (trainers) names
                trainers_elements = soup.find_all("div", class_="datapoint")
                trainer_count = 0
                for trainer_element in trainers_elements:
                    if "Director Técnico" in trainer_element.text:
                        trainer_name = trainer_element.text.split(":")[-1].strip()
                        if trainer_count == 0:
                            home_trainer = trainer_name
                            trainer_count += 1
                        elif trainer_count == 1:
                            away_trainer = trainer_name
                            trainer_count += 1
                if "home_trainer" not in locals():
                    print(f"Home coach not found for link {link}. Assigning NaN.")
                    home_trainer = "NaN"
                if "away_trainer" not in locals():
                    print(f"Away coach not found for link {link}. Assigning NaN.")
                    away_trainer = "NaN"
                # Remove accents and replace spaces with underscores
                home_trainer = unidecode(home_trainer).replace(" ", "_")
                away_trainer = unidecode(away_trainer).replace(" ", "_")
                # Assign the processed names to the DataFrame
                self.df_final.at[idx, "home_trainer"] = home_trainer
                self.df_final.at[idx, "away_trainer"] = away_trainer

                # List of stadiums without accents
                stadiums = [
                    "San_Mames",
                    "Estadio_de_Balaidos",
                    "Estadio_de_Mestalla",
                    "Iberostar_Estadi",
                    "Estadio_Municipal_de_Butarque",
                    "Estadio_de_la_Ceramica",
                    "Estadio_de_Mendizorroza",
                    "RCDE_Stadium",
                    "Estadio_Benito_Villamarin",
                    "Riyadh_Air_Metropolitan_Stadium",
                    "Estadio_Nuevo_Los_Carmenes",
                    "Estadio_Ciudad_de_Valencia",
                    "Estadio_El_Sadar",
                    "Estadio_Santiago_Bernabeu",
                    "Coliseum_Alfonso_Perez",
                    "Camp_Nou",
                    "Estadio_Ramon_Sanchez_Pizjuan",
                    "Estadio_Municipal_de_Anoeta",
                    "Estadio_Municipal_de_Ipurua",
                    "Estadio_Municipal_Jose_Zorrilla",
                    "Estadio_Alfredo_Di_Stefano",
                    "Estadio_Ramon_de_Carranza",
                    "Estadio_El_Alcoraz",
                    "Estadio_Manuel_Martinez_Valero",
                    "Estadio_Nuevo_Mirandilla",
                    "Estadio_del_Rayo_Vallecano",
                    "Power_Horse_Stadium",
                    "Estadio_Civitas_Metropolitano",
                    "Reale_Arena",
                    "Estadi_Municipal_de_Montilivi",
                    "Estadio_de_Gran_Canaria",
                    "Estadio_Abanca_Balaidos",
                    "Estadi_Mallorca_Son_Moix",
                    "Estadi_Olimpic_Lluis_Companys",
                ]
                # List of corresponding stadium capacities
                capacities = [
                    53289,
                    24870,
                    49430,
                    26020,
                    12454,
                    23500,
                    19840,
                    40500,
                    60721,
                    68456,
                    19189,
                    26354,
                    23576,
                    78297,
                    16500,
                    99354,
                    43883,
                    40000,
                    8164,
                    27618,
                    5600,
                    20724,
                    9100,
                    31388,
                    20724,
                    14708,
                    18331,
                    68456,
                    40000,
                    14624,
                    32400,
                    24870,
                    26020,
                    55926,
                ]
                # Create a dictionary of stadiums and their corresponding capacities
                stadium_capacity_dict = dict(zip(stadiums, capacities))
                # Extract stadium information
                stadium_element = soup.find("div", class_="scorebox_meta")
                if stadium_element:
                    stadium_info = stadium_element.find("strong", string="Sedes")
                    if stadium_info:
                        stadium = (
                            stadium_info.find_next("small")
                            .find_next("small")
                            .text.strip()
                        )
                    else:
                        print(f"Stadium not found for link {link}. Assigning NaN.")
                        stadium = "NaN"
                else:
                    print(f"Stadium not found for link {link}. Assigning NaN.")
                    stadium = "NaN"
                # Remove everything after the comma (including the comma itself)
                if "," in stadium:
                    stadium = stadium.split(",")[0].strip()
                # Remove accents and replace spaces with underscores
                stadium = unidecode(stadium).replace(" ", "_")
                # Assign the processed stadium name to the DataFrame
                self.df_final.at[idx, "stadium"] = stadium
                # Check if the stadium exists in the dictionary and assign the corresponding capacity
                if stadium in stadium_capacity_dict:
                    # Assign the corresponding capacity to the 'capacity' column
                    self.df_final.at[idx, "capacity"] = stadium_capacity_dict[stadium]
                else:
                    # If the stadium is not found in the dictionary, assign NaN
                    print(
                        f"Stadium '{stadium}' not found in dictionary. Assigning NaN."
                    )
                    self.df_final.at[idx, "capacity"] = np.nan

                # Extract attendance information
                attendance_element = soup.find("div", class_="scorebox_meta")
                if attendance_element:
                    attendance_info = attendance_element.find(
                        "strong", string="Asistencia"
                    )
                    if attendance_info:
                        attendance = (
                            attendance_info.find_next("small")
                            .find_next("small")
                            .text.strip()
                        )
                        try:
                            attendance = int(
                                attendance.replace(",", "").replace(".", "")
                            )
                        except ValueError:
                            attendance = 0
                else:
                    print(f"Attendance not found for link {link}. Assigning 0.")
                    attendance = 0

                # Assign attendance value to the DataFrame
                self.df_final.at[idx, "attendance"] = attendance

                # Calculate and assign attendance percentage (ensure capacity is not NaN)
                if (
                    pd.notna(self.df_final.at[idx, "capacity"])
                    and self.df_final.at[idx, "capacity"] > 0
                ):
                    attendance_percentage = (
                        attendance / self.df_final.at[idx, "capacity"]
                    )
                    # Ensure attendance% does not exceed 1
                    if attendance_percentage > 1:
                        attendance_percentage = 1
                    self.df_final.at[idx, "attendance%"] = attendance_percentage
                else:
                    self.df_final.at[idx, "attendance%"] = (
                        np.nan
                    )  # If capacity is NaN or 0, set attendance% as NaN
                
                # Extract referee information
                referee_element = soup.find("div", class_="scorebox_meta")
                if referee_element:
                    referee_info = referee_element.find_next(
                        "strong", string="Autoridades"
                    )
                    if referee_info:
                        referee_span = (
                            referee_info.find_next("small")
                            .find_next("small")
                            .find("span", style="display:inline-block")
                        )
                        if referee_span:
                            referee = referee_span.text.strip()
                            # Remove accents, replace spaces with underscores, and remove " (Arbitro)"
                            referee = (
                                unidecode(referee)
                                .replace(" ", "_")
                                .replace("_(Arbitro)", "")
                            )
                else:
                    print(f"Referee not found for link {link}. Assigning NaN.")
                    referee = "NaN"
                self.df_final.at[idx, "referee"] = referee

                # Extract VAR information
                var_element = soup.find("div", class_="scorebox_meta")
                if var_element:
                    var_info = var_element.find_next("strong", string="Autoridades")
                    if var_info:
                        var_span = (
                            var_info.find_next("small")
                            .find_next("small")
                            .find_next("span")
                            .find_next("span")
                            .find_next("span")
                            .find_next("span")
                            .find_next("span")
                        )
                        if var_span:
                            var = var_span.text.strip()
                            # Remove accents, replace spaces with underscores, and remove " (VAR)"
                            var = unidecode(var).replace(" ", "_").replace("_(VAR)", "")
                else:
                    print(f"VAR not found for link {link}. Assigning NaN.")
                    var = "NaN"
                self.df_final.at[idx, "var"] = var

                # Extract team lineups
                lineup_elements = soup.find_all(
                    "th", string=lambda text: text and "(" in text and ")" in text
                )
                if len(lineup_elements) >= 1:
                    home_match = re.search(r"\((.*?)\)", lineup_elements[0].text)
                    if home_match:
                        home_team_lineup = home_match.group(1)
                    else:
                        home_team_lineup = "NaN"
                else:
                    print(f"Home lineup not found for link {link}. Assigning NaN.")
                    home_team_lineup = "NaN"

                if len(lineup_elements) >= 2:
                    away_match = re.search(r"\((.*?)\)", lineup_elements[1].text)
                    if away_match:
                        away_team_lineup = away_match.group(1)
                    else:
                        away_team_lineup = "NaN"
                else:
                    print(f"Away lineup not found for link {link}. Assigning NaN.")
                    away_team_lineup = "NaN"

                self.df_final.at[idx, "home_team_lineup"] = home_team_lineup
                self.df_final.at[idx, "away_team_lineup"] = away_team_lineup

                # Posesiond de balon
                team_stats = soup.find("div", id="team_stats")

                if team_stats:
                    possession_values = team_stats.find_all("strong")

                    if len(possession_values) >= 2:
                        home_possession = possession_values[0].text.strip().strip("%")
                        away_possession = possession_values[1].text.strip().strip("%")

                        self.df_final.at[idx, "home_possession"] = (
                            float(home_possession) if home_possession else None
                        )
                        self.df_final.at[idx, "away_possession"] = (
                            float(away_possession) if away_possession else None
                        )
                    else:
                        print(f"Possession values not found for link {link}. Assigning NaN.")
                else:
                    print(f"Team stats section not found for link {link}. Assigning NaN.")

                # Remove rows that are completely empty
                self.df_final = self.df_final.dropna(how="all")

            except Exception as e:
                print(f"Error processing link {link}: {e}")
                continue
        
            time.sleep(6)
        self.df_matches = self.df_final
        return self.df_matches

    def save_to_csv(self):
        """
        Save the processed data into a CSV file, appending new rows if the file already exists.

        Args:
            season (str): The season to be included in the filename.
        """
        try:
            # Check if the file exists
            if os.path.exists(self.filename):
                # If the file exists, read the existing CSV into a DataFrame
                existing_df = pd.read_csv(self.filename)
                # Append the new rows from self.df_matches to the existing DataFrame
                combined_df = pd.concat([existing_df, self.df_matches], ignore_index=True)
                # Save the combined DataFrame back to the CSV file
                combined_df.to_csv(self.filename, index=False)
                print(f"File updated successfully as {self.filename}")

            else:
                self.df_matches.to_csv(self.filename, index=False)
                print(f"File saved successfully as {self.filename}")

        except Exception as e:
            print(f"Error saving file {self.filename}: {e}")

    def run(self):
        """
        Execute the full process: get links, get gameweeks, get statistics, and save to CSV files for both teams.
        """
        print(f"Starting to collect matches data...")

        # Step 1: Extract the season
        self.url = "https://fbref.com/es/comps/12/horario/Resultados-y-partidos-en-La-Liga"
        self.filename = f"../database/{self.season}/matches_{self.season}.csv"
        
        if self.season:
            print(f"Season extracted: {self.season}")
        else:
            print("Season not found in the URL. Using default value 'unknown_season'.")

        # Step 2: Get all the links to the match pages from the provided URL

        # Step 3: Get the gameweek data from the provided URL

        # Step 4: Create a dataframe with match details, such as teams, dates, and other match-related information
        self.get_match_data()

        # Step 5: Retrieve statistics for each match, such as goals, assists, and other relevant data
        self.get_statistics()

        # Step 6: Save the processed data into a CSV file with the season name in the filename
        self.save_to_csv()

        print(f"Collecting matches data process completed successfully!")

## Obtaining players and keeper data from the website

In [3]:
class Players_data:
    def __init__(self, last_season, choice_1):
        """
        Initializes the Players_data with links and gameweeks files.
        - links_file: Path to the CSV file containing match links.
        - gameweeks_file: Path to the CSV file containing gameweek data.
        """
        self.last_season = last_season
        self.choice_1 = choice_1  
        self.soup = None  # Initialize BeautifulSoup object as None
        self.teams_data = {}  # Dictionary to store team information
        self.filename = None

    def fetch_page(self):
        """
        Fetches the web page content from the provided URL and initializes BeautifulSoup.
        Adds a delay to prevent overloading the server.
        """
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:131.0) Gecko/20100101 Firefox/131.0",
            "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8",
            "Accept-Language": "en-US,en;q=0.5",
            "Accept-Encoding": "gzip, deflate, br",
            "Connection": "keep-alive",
            "Upgrade-Insecure-Requests": "1",
            "DNT": "1",
        }
        response = requests.get(self.link, headers=headers)

        if response.status_code != 200:
            raise Exception(f"Error accessing the page: {response.status_code}")

        self.soup = BeautifulSoup(response.content, "html.parser")
        time.sleep(6)

    def extract_teams_ids(self):
        """
        Extracts the IDs and names of the home and away teams using team logos.
        """
        # Verifica que el link esté definido antes de intentar acceder a la página
        if not hasattr(self, "link") or self.link is None:
            raise Exception("Link not defined for the match.")

        self.fetch_page()  # Fetch the web page content for the provided URL

        # Buscar las imágenes de los equipos
        team_imgs = self.soup.find_all("img", class_="teamlogo", src=True)

        # Verifica si se encontraron al menos dos imágenes de los equipos
        if len(team_imgs) >= 2:
            self.teams_data = {
                "home": {
                    "id": team_imgs[0]["src"].split("/")[-1].split(".")[0],
                    "name": team_imgs[0]["alt"]
                    .replace(" Club Crest", "")
                    .replace(" ", "_"),
                },
                "away": {
                    "id": team_imgs[1]["src"].split("/")[-1].split(".")[0],
                    "name": team_imgs[1]["alt"]
                    .replace(" Club Crest", "")
                    .replace(" ", "_"),
                },
            }
        else:
            raise Exception(
                "Not enough team logos found. Expected at least two team logos."
            )

    def extract_players_table(
        self, team_type, table_type, header_offset, columns_to_drop
    ):
        """
        Extracts a specific player statistics table for the given team and table type.
        - team_type: "home" or "away".
        - table_type: Type of the table (e.g., "summary", "passing").
        - header_offset: Number of header columns to skip.
        - columns_to_drop: List of columns to drop from the table.
        """
        # Get the team ID based on the team type (home or away)
        team_id = self.teams_data[team_type]["id"]

        # Construct the CSS selector for the specific table
        players_table_selector = f"#div_stats_{team_id}_{table_type}"

        # Select the table element using the constructed selector
        table = self.soup.select_one(players_table_selector)

        # Check if the table exists
        if not table:
            raise Exception(
                f"Payers table {table_type} not found for team {team_type}."
            )

        # Extract headers from the table, skipping the specified number of columns
        headers = [th.text.strip() for th in table.find("thead").find_all("th")][
            header_offset:
        ]

        # Extract rows of data from the table body
        rows = [
            [cell.text.strip() for cell in row.find_all(["td", "th"])]
            for row in table.find("tbody").find_all("tr")
        ]

        # Create a DataFrame from the extracted rows and headers
        df = pd.DataFrame(rows, columns=headers)

        # Drop unnecessary columns specified in the columns_to_drop list
        df = df.loc[:, ~df.columns.isin(columns_to_drop)]

        return df

        # Add a delay to avoid overloading the server
        time.sleep(6)

    def process_players_data(self, team_type):
        """
        Processes all player statistics tables for a specific team (home or away).
        Combines data from multiple table types into a dictionary of DataFrames.
        """
        # Define columns to drop for each table type
        columns_to_drop = {
            "summary": [
                "Gls",
                "Ass",
                "TP",
                "TPint",
                "TA",
                "TR",
                "Toques",
                "Tkl",
                "Int",
                "Bloqueos",
                "xG",
                "npxG",
                "xAG",
                "ACT",
                "ACG",
                "Cmp",
                "Int.",
                "% Cmp",
                "PrgP",
                "Transportes",
                "PrgC",
                "Att",
                "Succ",
            ],
            "passing": ["Jugador", "núm.", "País", "Posc", "Edad", "Mín"],
            "passing_types": [
                "Jugador",
                "núm.",
                "País",
                "Posc",
                "Edad",
                "Mín",
                "Int.",
                "Cmp",
            ],
            "defense": ["Jugador", "núm.", "País", "Posc", "Edad", "Mín"],
            "possession": [
                "Jugador",
                "núm.",
                "País",
                "Posc",
                "Edad",
                "Mín",
                "Tkld",
                "Tkld%",
            ],
            "misc": [
                "Jugador",
                "núm.",
                "País",
                "Posc",
                "Edad",
                "Mín",
                "Pcz",
                "PA",
                "Int",
                "TklG",
                "GC",
            ],
        }

        # Initialize an empty dictionary to store DataFrames for each table type
        tables = {}

        # Loop through each table type and extract its data
        for table_type, header_offset in [
            ("summary", 7),
            ("passing", 9),
            ("passing_types", 4),
            ("defense", 5),
            ("possession", 5),
            ("misc", 3),
        ]:
            tables[table_type] = self.extract_players_table(
                team_type,
                table_type,
                header_offset,
                columns_to_drop.get(table_type, []),
            )

        return tables

    def save_players_tables(self, match, season):
        """
        Processes and saves player statistics tables for both home and away teams.
        Combines data from all table types and writes the final table to a CSV file.
        """
        # Extract IDs and names of the teams
        self.extract_teams_ids()

        # Process data for both home and away teams
        for team_type in ["home", "away"]:
            # Get the team name
            team_name = self.teams_data[team_type]["name"]

            # Extract and process all player statistics tables for the team
            team_tables = self.process_players_data(team_type)

            # Combine all extracted tables into a single DataFrame
            final_table = pd.concat(team_tables.values(), axis=1)

            # Define the new column names for the dataset with the team_type prefix
            new_columns = [
                f"{team_type}_Players",
                f"{team_type}_Number",
                f"{team_type}_Nationality",
                f"{team_type}_Position",
                f"{team_type}_PlayersAge",
                f"{team_type}_PlayersMinutes",
                f"{team_type}_PlayersGoals",
                f"{team_type}_PlayersShots",
                f"{team_type}_PlayersShotsOnTarget",
                f"{team_type}_PlayersCompletedPasses",
                f"{team_type}_PlayersAttemptedPasses",
                f"{team_type}_Players%CompletedPasses",
                f"{team_type}_PlayersDistancePasses",
                f"{team_type}_PlayersDistanceProgression",
                f"{team_type}_PlayersShortPasses",
                f"{team_type}_PlayersAttemptedShortPasses",
                f"{team_type}_Players%ShortCompletedPasses",
                f"{team_type}_PlayersMediumPasses",
                f"{team_type}_PlayersAttemptedMediumPasses",
                f"{team_type}_Players%MediumCompletedPasses",
                f"{team_type}_PlayersLongPasses",
                f"{team_type}_PlayersAttemptedLongPasses",
                f"{team_type}_Players%LongCompletedPasses",
                f"{team_type}_PlayersAssistance",
                f"{team_type}_PlayersExpectedGoalsAssistance",
                f"{team_type}_PlayersExpectedAssistance",
                f"{team_type}_PlayersKeyPasses",
                f"{team_type}_PlayersLast1/3Passes",
                f"{team_type}_PlayersGoalAreaPasses",
                f"{team_type}_PlayersGoalAreaCrosses",
                f"{team_type}_PlayersGoalPasses",
                f"{team_type}_PlayersLiveBallPasses",
                f"{team_type}_PlayersDeadBallPasses",
                f"{team_type}_PlayersFreeKick",
                f"{team_type}_PlayersLongPasses",
                f"{team_type}_PlayersSidePasses",
                f"{team_type}_PlayersCrosses",
                f"{team_type}_PlayersStrongcrosses",
                f"{team_type}_PlayersCorner",
                f"{team_type}_PlayersCornerIn",
                f"{team_type}_PlayersCornerOut",
                f"{team_type}_PlayersCornerRect",
                f"{team_type}_PlayersOffsidePasses",
                f"{team_type}_PlayersPassesBlocked",
                f"{team_type}_PlayersTackles",
                f"{team_type}_PlayersSuccessfulTackles",
                f"{team_type}_PlayersTacklesInDefense",
                f"{team_type}_PlayersTacklesInMedium",
                f"{team_type}_PlayersTacklesInAttack",
                f"{team_type}_PlayersDribblerTackles",
                f"{team_type}_PlayersAttemptedDribblerTackles",
                f"{team_type}_Players%DribblerTacklesCompleted",
                f"{team_type}_PlayersDribblerTacklesNonCompleted",
                f"{team_type}_PlayersBallsBlocked",
                f"{team_type}_PlayersShotsBlocked",
                f"{team_type}_PlayersPassesBlocked",
                f"{team_type}_PlayersInterceptions",
                f"{team_type}_PlayersTackles+Interceptions",
                f"{team_type}_PlayersClearances",
                f"{team_type}_PlayersMistakesRivalShots",
                f"{team_type}_PlayersTouches",
                f"{team_type}_PlayersOwnPenaltyAreaTouches",
                f"{team_type}_PlayersTouchesInDefense",
                f"{team_type}_PlayersTouchesInMedium",
                f"{team_type}_PlayersTouchesInAttack",
                f"{team_type}_PlayersAwayPenaltyAreaTouches",
                f"{team_type}_PlayersLiveBallTouches",
                f"{team_type}_PlayersAttemptedDribbles",
                f"{team_type}_PlayersDribblesCompleted",
                f"{team_type}_Players%DribblesCompleted",
                f"{team_type}_PlayersBallCarries",
                f"{team_type}_PlayersDistanceCarried",
                f"{team_type}_PlayersForwardDistanceCarried",
                f"{team_type}_PlayersForwardCarries",
                f"{team_type}_PlayersCarriesInAttack",
                f"{team_type}_PlayersAwayPenaltyAreaCarries",
                f"{team_type}_PlayersLostControlCarries",
                f"{team_type}_PlayersLostCarries",
                f"{team_type}_PlayersPassesReception",
                f"{team_type}_PlayersAttackPassesReception",
                f"{team_type}_PlayersYellowCards",
                f"{team_type}_PlayersRedCards",
                f"{team_type}_PlayersSecondYellowCards",
                f"{team_type}_PlayersFouls",
                f"{team_type}_PlayersFoulsReceived",
                f"{team_type}_PlayersPenalties",
                f"{team_type}_PlayersPenaltiesConceded",
                f"{team_type}_PlayersLostBallRecoveries",
                f"{team_type}_PlayersAerialsWon",
                f"{team_type}_PlayersAerialsLost",
                f"{team_type}_Players%AerialsWon",
            ]

            # Rename the columns of the DataFrame
            final_table.columns = new_columns

            # Convert to integer if the value is a string with digits, otherwise set to None
            final_table[f"{team_type}_PlayersAge"] = final_table[f"{team_type}_PlayersAge"].apply(
                lambda x: int(x[:2]) if isinstance(x, str) and x[:2].isdigit() else None
            )

            # Remove rows where PlayersAge is None
            final_table = final_table.dropna(subset=[f"{team_type}_PlayersAge"])

            # Define columns to calculate the mean
            columns_to_mean = [
                f"{team_type}_PlayersAge",
                f"{team_type}_Players%CompletedPasses",
                f"{team_type}_Players%ShortCompletedPasses",
                f"{team_type}_Players%MediumCompletedPasses",
                f"{team_type}_Players%LongCompletedPasses",
                f"{team_type}_Players%DribblerTacklesCompleted",
                f"{team_type}_Players%DribblesCompleted",
                f"{team_type}_Players%AerialsWon",
            ]

            # Define columns to calculate the sum
            columns_to_sum = [
                f"{team_type}_PlayersMinutes",
                f"{team_type}_PlayersGoals",
                f"{team_type}_PlayersShots",
                f"{team_type}_PlayersShotsOnTarget",
                f"{team_type}_PlayersCompletedPasses",
                f"{team_type}_PlayersAttemptedPasses",
                f"{team_type}_PlayersDistancePasses",
                f"{team_type}_PlayersDistanceProgression",
                f"{team_type}_PlayersShortPasses",
                f"{team_type}_PlayersAttemptedShortPasses",
                f"{team_type}_PlayersMediumPasses",
                f"{team_type}_PlayersAttemptedMediumPasses",
                f"{team_type}_PlayersLongPasses",
                f"{team_type}_PlayersAttemptedLongPasses",
                f"{team_type}_PlayersAssistance",
                f"{team_type}_PlayersExpectedGoalsAssistance",
                f"{team_type}_PlayersExpectedAssistance",
                f"{team_type}_PlayersKeyPasses",
                f"{team_type}_PlayersLast1/3Passes",
                f"{team_type}_PlayersGoalAreaPasses",
                f"{team_type}_PlayersGoalAreaCrosses",
                f"{team_type}_PlayersGoalPasses",
                f"{team_type}_PlayersLiveBallPasses",
                f"{team_type}_PlayersDeadBallPasses",
                f"{team_type}_PlayersFreeKick",
                f"{team_type}_PlayersLongPasses",
                f"{team_type}_PlayersSidePasses",
                f"{team_type}_PlayersCrosses",
                f"{team_type}_PlayersStrongcrosses",
                f"{team_type}_PlayersCorner",
                f"{team_type}_PlayersCornerIn",
                f"{team_type}_PlayersCornerOut",
                f"{team_type}_PlayersCornerRect",
                f"{team_type}_PlayersOffsidePasses",
                f"{team_type}_PlayersPassesBlocked",
                f"{team_type}_PlayersTackles",
                f"{team_type}_PlayersSuccessfulTackles",
                f"{team_type}_PlayersTacklesInDefense",
                f"{team_type}_PlayersTacklesInMedium",
                f"{team_type}_PlayersTacklesInAttack",
                f"{team_type}_PlayersDribblerTackles",
                f"{team_type}_PlayersAttemptedDribblerTackles",
                f"{team_type}_PlayersDribblerTacklesNonCompleted",
                f"{team_type}_PlayersBallsBlocked",
                f"{team_type}_PlayersShotsBlocked",
                f"{team_type}_PlayersPassesBlocked",
                f"{team_type}_PlayersInterceptions",
                f"{team_type}_PlayersTackles+Interceptions",
                f"{team_type}_PlayersClearances",
                f"{team_type}_PlayersMistakesRivalShots",
                f"{team_type}_PlayersTouches",
                f"{team_type}_PlayersOwnPenaltyAreaTouches",
                f"{team_type}_PlayersTouchesInDefense",
                f"{team_type}_PlayersTouchesInMedium",
                f"{team_type}_PlayersTouchesInAttack",
                f"{team_type}_PlayersAwayPenaltyAreaTouches",
                f"{team_type}_PlayersLiveBallTouches",
                f"{team_type}_PlayersAttemptedDribbles",
                f"{team_type}_PlayersDribblesCompleted",
                f"{team_type}_PlayersBallCarries",
                f"{team_type}_PlayersDistanceCarried",
                f"{team_type}_PlayersForwardDistanceCarried",
                f"{team_type}_PlayersForwardCarries",
                f"{team_type}_PlayersCarriesInAttack",
                f"{team_type}_PlayersAwayPenaltyAreaCarries",
                f"{team_type}_PlayersLostControlCarries",
                f"{team_type}_PlayersLostCarries",
                f"{team_type}_PlayersPassesReception",
                f"{team_type}_PlayersAttackPassesReception",
                f"{team_type}_PlayersYellowCards",
                f"{team_type}_PlayersRedCards",
                f"{team_type}_PlayersSecondYellowCards",
                f"{team_type}_PlayersFouls",
                f"{team_type}_PlayersFoulsReceived",
                f"{team_type}_PlayersPenalties",
                f"{team_type}_PlayersPenaltiesConceded",
                f"{team_type}_PlayersLostBallRecoveries",
                f"{team_type}_PlayersAerialsWon",
                f"{team_type}_PlayersAerialsLost",
            ]

            # Convert the mean columns to numeric
            for col in columns_to_mean:
                final_table[col] = final_table[col].apply(
                    pd.to_numeric, errors="coerce"
                )

            # Convert the sum columns to numeric
            for col in columns_to_sum:
                final_table[col] = final_table[col].apply(
                    pd.to_numeric, errors="coerce"
                )

            # Calculate the mean and sum for specified columns
            mean_values = final_table[columns_to_mean].mean()
            sum_values = final_table[columns_to_sum].sum()

            # Create a new row for totals with placeholder values
            total_row = {col: "-" for col in final_table.columns}

            # Populate the total row with mean values
            for col, mean in mean_values.items():
                total_row[col] = mean

            # Populate the total row with sum values
            for col, total in sum_values.items():
                total_row[col] = total

            # Add the number of rows (lines) to the first column of the total row
            num_lines = len(final_table)
            total_row[final_table.columns[0]] = num_lines

            # Check if the 'id' column exists, if not, create it with NaN values
            if "id" not in final_table.columns:
                final_table["id"] = np.nan  # Create the column with NaN values

            # Add the match ID to the total row
            total_row["id"] = match

            # Append the total row to the DataFrame
            final_table.loc[len(final_table)] = total_row

            # Define the columns to append to the existing CSV
            columns_to_append = [
                f"{team_type}_Players",
                f"{team_type}_PlayersAge",
                f"{team_type}_PlayersMinutes",
                f"{team_type}_PlayersShots",
                f"{team_type}_PlayersShotsOnTarget",
                f"{team_type}_PlayersCompletedPasses",
                f"{team_type}_PlayersAttemptedPasses",
                f"{team_type}_Players%CompletedPasses",
                f"{team_type}_PlayersDistancePasses",
                f"{team_type}_PlayersDistanceProgression",
                f"{team_type}_PlayersShortPasses",
                f"{team_type}_PlayersAttemptedShortPasses",
                f"{team_type}_Players%ShortCompletedPasses",
                f"{team_type}_PlayersMediumPasses",
                f"{team_type}_PlayersAttemptedMediumPasses",
                f"{team_type}_Players%MediumCompletedPasses",
                f"{team_type}_PlayersLongPasses",
                f"{team_type}_PlayersAttemptedLongPasses",
                f"{team_type}_Players%LongCompletedPasses",
                f"{team_type}_PlayersAssistance",
                f"{team_type}_PlayersExpectedGoalsAssistance",
                f"{team_type}_PlayersExpectedAssistance",
                f"{team_type}_PlayersKeyPasses",
                f"{team_type}_PlayersLast1/3Passes",
                f"{team_type}_PlayersGoalAreaPasses",
                f"{team_type}_PlayersGoalAreaCrosses",
                f"{team_type}_PlayersGoalPasses",
                f"{team_type}_PlayersLiveBallPasses",
                f"{team_type}_PlayersDeadBallPasses",
                f"{team_type}_PlayersFreeKick",
                f"{team_type}_PlayersLongPasses",
                f"{team_type}_PlayersSidePasses",
                f"{team_type}_PlayersCrosses",
                f"{team_type}_PlayersStrongcrosses",
                f"{team_type}_PlayersCorner",
                f"{team_type}_PlayersCornerIn",
                f"{team_type}_PlayersCornerOut",
                f"{team_type}_PlayersCornerRect",
                f"{team_type}_PlayersOffsidePasses",
                f"{team_type}_PlayersPassesBlocked",
                f"{team_type}_PlayersTackles",
                f"{team_type}_PlayersSuccessfulTackles",
                f"{team_type}_PlayersTacklesInDefense",
                f"{team_type}_PlayersTacklesInMedium",
                f"{team_type}_PlayersTacklesInAttack",
                f"{team_type}_PlayersDribblerTackles",
                f"{team_type}_PlayersAttemptedDribblerTackles",
                f"{team_type}_Players%DribblerTacklesCompleted",
                f"{team_type}_PlayersDribblerTacklesNonCompleted",
                f"{team_type}_PlayersBallsBlocked",
                f"{team_type}_PlayersShotsBlocked",
                f"{team_type}_PlayersPassesBlocked",
                f"{team_type}_PlayersInterceptions",
                f"{team_type}_PlayersTackles+Interceptions",
                f"{team_type}_PlayersClearances",
                f"{team_type}_PlayersMistakesRivalShots",
                f"{team_type}_PlayersTouches",
                f"{team_type}_PlayersOwnPenaltyAreaTouches",
                f"{team_type}_PlayersTouchesInDefense",
                f"{team_type}_PlayersTouchesInMedium",
                f"{team_type}_PlayersTouchesInAttack",
                f"{team_type}_PlayersAwayPenaltyAreaTouches",
                f"{team_type}_PlayersLiveBallTouches",
                f"{team_type}_PlayersAttemptedDribbles",
                f"{team_type}_PlayersDribblesCompleted",
                f"{team_type}_Players%DribblesCompleted",
                f"{team_type}_PlayersBallCarries",
                f"{team_type}_PlayersDistanceCarried",
                f"{team_type}_PlayersForwardDistanceCarried",
                f"{team_type}_PlayersForwardCarries",
                f"{team_type}_PlayersCarriesInAttack",
                f"{team_type}_PlayersAwayPenaltyAreaCarries",
                f"{team_type}_PlayersLostControlCarries",
                f"{team_type}_PlayersLostCarries",
                f"{team_type}_PlayersPassesReception",
                f"{team_type}_PlayersAttackPassesReception",
                f"{team_type}_PlayersYellowCards",
                f"{team_type}_PlayersRedCards",
                f"{team_type}_PlayersSecondYellowCards",
                f"{team_type}_PlayersFouls",
                f"{team_type}_PlayersFoulsReceived",
                f"{team_type}_PlayersPenalties",
                f"{team_type}_PlayersPenaltiesConceded",
                f"{team_type}_PlayersLostBallRecoveries",
                f"{team_type}_PlayersAerialsWon",
                f"{team_type}_PlayersAerialsLost",
                f"{team_type}_Players%AerialsWon",
            ]

            # Extract the last row from final_table (this contains the sums and means)
            last_row = final_table.iloc[-1][columns_to_append]

            # Load the existing CSV file from the parent directory
            existing_df = pd.read_csv(self.filename)

            # Get the ID from the last row of final_table
            last_row_id = final_table.iloc[-1]["id"]

            # Find the row in the existing CSV based on the ID
            row_index = existing_df[existing_df["id"] == last_row_id].index

            # Check if the row exists
            if row_index.empty:
                raise ValueError(
                    f"Error: No row with ID {last_row_id} found in {self.filename}"
                )

            # Check if the columns exist in the existing CSV, if not, create them after the existing columns
            for column in columns_to_append:
                if column not in existing_df.columns:
                    existing_df[column] = pd.NA  # Create the column with missing values

            # Ensure the new columns are placed after the existing columns
            existing_columns = existing_df.columns.tolist()
            new_columns = [
                column for column in columns_to_append if column not in existing_columns
            ]
            existing_df = existing_df[existing_columns + new_columns]

            # Update the row with the new data from last_row
            for column, value in last_row.items():
                existing_df.at[row_index[0], column] = value

            # Save the updated DataFrame back to the CSV
            existing_df.to_csv(self.filename, index=False)

            time.sleep(6)  # Sleep to avoid making too many requests in a short time

    def extract_keeper_table(self, team_type, header_offset):
        """
        Extracts a specific keeper statistics table for the given team.
        - team_type: "home" or "away".
        - header_offset: Number of header columns to skip.
        """
        # Get the team ID based on the team type (home or away)
        team_id = self.teams_data[team_type]["id"]

        # Construct the CSS selector for the specific table
        keeper_table_selector = f"#keeper_stats_{team_id}"

        # Select the table element using the constructed selector
        table = self.soup.select_one(keeper_table_selector)

        # Check if the table exists
        if not table:
            raise Exception(f"Keeper table not found for team {team_type}.")

        # Extract headers from the table, skipping the specified number of columns
        headers = [th.text.strip() for th in table.find("thead").find_all("th")][
            header_offset:
        ]

        # Extract rows of data from the table body
        rows = [
            [cell.text.strip() for cell in row.find_all(["td", "th"])]
            for row in table.find("tbody").find_all("tr")
        ]

        # Create a DataFrame from the extracted rows and headers
        df = pd.DataFrame(rows, columns=headers)

        return df

        # Add a delay to avoid overloading the server
        time.sleep(6)

    def process_keeper_data(self, team_type):
        """
        Processes all keeper statistics tables for a specific team (home or away).
        Combines data into a dictionary of DataFrames.
        """
        # Extract data from the keeper statistics table
        table = self.extract_keeper_table(team_type, header_offset=7)

        return table

    def save_keeper_tables(self, match, season):
        """
        Processes and saves keeper statistics table for both home and away teams.
        Writes the final table to a CSV file.
        """
        # Extract IDs and names of the teams
        self.extract_teams_ids()

        # Process data for both home and away teams
        for team_type in ["home", "away"]:

            # Extract and process all keeper statistics tables for the team
            final_table = self.process_keeper_data(team_type)

            # Define the new column names for the dataset
            new_columns = [
                f"{team_type}_KeepersKeepers",
                f"{team_type}_KeepersNationality",
                f"{team_type}_KeepersAge",
                f"{team_type}_KeepersMinutes",
                f"{team_type}_KeepersShotsOnTargetAgainst",
                f"{team_type}_KeepersGoalsAgainst",
                f"{team_type}_KeepersSaved",
                f"{team_type}_Keepers%Saved",
                f"{team_type}_KeepersxG",
                f"{team_type}_KeepersPassesLaunched",
                f"{team_type}_KeepersAttemptedPassesLaunched",
                f"{team_type}_Keepers%CompletedPassesLaunched",
                f"{team_type}_KeepersPasses",
                f"{team_type}_KeepersAttemptedPasses",
                f"{team_type}_Keepers%CompletedPasses",
                f"{team_type}_KeepersPassesDistance",
                f"{team_type}_KeepersAttemptedKicks",
                f"{team_type}_Keepers%Kicks",
                f"{team_type}_KeepersKicksDistance",
                f"{team_type}_KeepersCrosses",
                f"{team_type}_KeepersCrossesStopped",
                f"{team_type}_Keepers%CrossesStopped",
                f"{team_type}_KeepersActionsOutsideArea",
                f"{team_type}_KeepersDistanceActionsArea",
            ]

            # Rename the columns of the DataFrame
            final_table.columns = new_columns

            # Convert the 'Age' column to integer by extracting the first two characters
            final_table[f"{team_type}_KeepersAge"] = final_table[
                f"{team_type}_KeepersAge"
            ].apply(lambda x: int(x[:2]) if isinstance(x, str) else 0)

            # Define columns to calculate the mean
            columns_to_mean = [
                f"{team_type}_KeepersAge",
                f"{team_type}_Keepers%Saved",
                f"{team_type}_Keepers%CompletedPassesLaunched",
                f"{team_type}_Keepers%CompletedPasses",
                f"{team_type}_KeepersPassesDistance",
                f"{team_type}_Keepers%Kicks",
                f"{team_type}_KeepersKicksDistance",
                f"{team_type}_Keepers%CrossesStopped",
                f"{team_type}_KeepersDistanceActionsArea",
            ]

            # Define columns to calculate the sum
            columns_to_sum = [
                f"{team_type}_KeepersKeepers",
                f"{team_type}_KeepersMinutes",
                f"{team_type}_KeepersShotsOnTargetAgainst",
                f"{team_type}_KeepersGoalsAgainst",
                f"{team_type}_KeepersSaved",
                f"{team_type}_KeepersxG",
                f"{team_type}_KeepersPassesLaunched",
                f"{team_type}_KeepersAttemptedPassesLaunched",
                f"{team_type}_KeepersPasses",
                f"{team_type}_KeepersAttemptedPasses",
                f"{team_type}_KeepersAttemptedKicks",
                f"{team_type}_KeepersCrosses",
                f"{team_type}_KeepersCrossesStopped",
                f"{team_type}_KeepersActionsOutsideArea",
            ]

            # Convert the mean columns to numeric
            for col in columns_to_mean:
                final_table[col] = final_table[col].apply(
                    pd.to_numeric, errors="coerce"
                )

            # Convert the sum columns to numeric
            for col in columns_to_sum:
                final_table[col] = final_table[col].apply(
                    pd.to_numeric, errors="coerce"
                )

            # Calculate the mean and sum for specified columns
            mean_values = final_table[columns_to_mean].mean()
            sum_values = final_table[columns_to_sum].sum()

            # Create a new row for totals with placeholder values
            total_row = {col: "-" for col in final_table.columns}

            # Populate the total row with mean values
            for col, mean in mean_values.items():
                total_row[col] = mean

            # Populate the total row with sum values
            for col, total in sum_values.items():
                total_row[col] = total

            # Add the number of rows (lines) to the first column of the total row
            num_lines = len(final_table)
            total_row[final_table.columns[0]] = num_lines

            # Check if the 'id' column exists, if not, create it with NaN values
            if "id" not in final_table.columns:
                final_table["id"] = np.nan  # Create the column with NaN values

            # Add the match ID to the total row
            total_row["id"] = match

            # Append the total row to the DataFrame
            final_table.loc[len(final_table)] = total_row

            # Define the columns to append to the existing CSV
            columns_to_append = [
                f"{team_type}_KeepersKeepers",
                f"{team_type}_KeepersMinutes",
                f"{team_type}_KeepersShotsOnTargetAgainst",
                f"{team_type}_KeepersGoalsAgainst",
                f"{team_type}_KeepersSaved",
                f"{team_type}_Keepers%Saved",
                f"{team_type}_KeepersxG",
                f"{team_type}_KeepersPassesLaunched",
                f"{team_type}_KeepersAttemptedPassesLaunched",
                f"{team_type}_Keepers%CompletedPassesLaunched",
                f"{team_type}_KeepersPasses",
                f"{team_type}_KeepersAttemptedPasses",
                f"{team_type}_Keepers%CompletedPasses",
                f"{team_type}_KeepersPassesDistance",
                f"{team_type}_KeepersAttemptedKicks",
                f"{team_type}_Keepers%Kicks",
                f"{team_type}_KeepersKicksDistance",
                f"{team_type}_KeepersCrosses",
                f"{team_type}_KeepersCrossesStopped",
                f"{team_type}_Keepers%CrossesStopped",
                f"{team_type}_KeepersActionsOutsideArea",
                f"{team_type}_KeepersDistanceActionsArea",
            ]

            # Extract the last row from final_table (this contains the sums and means)
            last_row = final_table.iloc[-1][columns_to_append]

            # Load the existing CSV file from the parent directory
            existing_df = pd.read_csv(self.filename)

            # Get the ID from the last row of final_table
            last_row_id = final_table.iloc[-1]["id"]

            # Find the row in the existing CSV based on the ID
            row_index = existing_df[existing_df["id"] == last_row_id].index

            # Check if the row exists
            if row_index.empty:
                raise ValueError(
                    f"Error: No row with ID {last_row_id} found in {self.filename}"
                )

            # Check if the columns exist in the existing CSV, if not, create them after the existing columns
            for column in columns_to_append:
                if column not in existing_df.columns:
                    existing_df[column] = pd.NA  # Create the column with missing values

            # Ensure the new columns are placed after the existing columns
            existing_columns = existing_df.columns.tolist()
            new_columns = [
                column for column in columns_to_append if column not in existing_columns
            ]
            existing_df = existing_df[existing_columns + new_columns]

            # Update the row with the new data from last_row
            for column, value in last_row.items():
                existing_df.at[row_index[0], column] = value

            # Save the updated DataFrame back to the CSV
            existing_df.to_csv(self.filename, index=False)

            time.sleep(6)  # Sleep to avoid making too many requests in a short time

    def run(self):
        """
        Processes multiple gameweek URLs by reading from a file of links and gameweeks.
        For each URL, extracts and saves player and keeper statistics tables.
        """
        print(f"Starting collecting players data...")

        self.filename = f"../database/{self.last_season}/matches_{self.last_season}.csv"
        filename = pd.read_csv(self.filename)

        start_index = len(filename) - self.choice_1

        for index in range(start_index, len(filename)):
            row = filename.iloc[index]
            link = row["link"]
            gameweek = row["gameweek"]

            line = index + 1

            print(f"Processing match {line}: {link}")

            self.link = link
            self.gameweek = gameweek

            self.extract_teams_ids()
            self.save_players_tables(line, self.last_season)
            self.save_keeper_tables(line, self.last_season)

            time.sleep(6)

        print(f"Collecting players data process completed successfully!")

## Making ranking

In [4]:
class FootballRanking:
    def __init__(self, last_season):
        self.last_season = last_season
        self.df = None  # Read the CSV file
        self.team_stats = {}  # Stores points, goals for, goals against, matches played, and card counts

    def update_team_stats(self, home_team, away_team, home_goals, away_goals, 
                      home_yellow_cards, home_red_cards, home_second_yellow_cards,
                      away_yellow_cards, away_red_cards, away_second_yellow_cards):
        """Update the stats for both teams after a match."""
        
        # Initialize stats if they don't exist for home team
        if home_team not in self.team_stats:
            self.team_stats[home_team] = {
                "points": 0, "goals_for": 0, "goals_against": 0, 
                "matches_played": 0, "yellow_cards": 0, "red_cards": 0, 
                "second_yellow_cards": 0
            }

        # Initialize stats if they don't exist for away team
        if away_team not in self.team_stats:
            self.team_stats[away_team] = {
                "points": 0, "goals_for": 0, "goals_against": 0, 
                "matches_played": 0, "yellow_cards": 0, "red_cards": 0, 
                "second_yellow_cards": 0
            }
        
        # Calculate points based on the match result
        if home_goals > away_goals:  # Home team wins
            home_points = 3
            away_points = 0
        elif home_goals < away_goals:  # Away team wins
            home_points = 0
            away_points = 3
        else:  # Draw
            home_points = 1
            away_points = 1
        
        # Update stats for home team
        self.team_stats[home_team]["points"] += home_points
        self.team_stats[home_team]["goals_for"] += home_goals
        self.team_stats[home_team]["goals_against"] += away_goals
        self.team_stats[home_team]["yellow_cards"] += home_yellow_cards
        self.team_stats[home_team]["red_cards"] += home_red_cards
        self.team_stats[home_team]["second_yellow_cards"] += home_second_yellow_cards
        self.team_stats[home_team]["matches_played"] += 1
        
        # Update stats for away team
        self.team_stats[away_team]["points"] += away_points
        self.team_stats[away_team]["goals_for"] += away_goals
        self.team_stats[away_team]["goals_against"] += home_goals
        self.team_stats[away_team]["yellow_cards"] += away_yellow_cards
        self.team_stats[away_team]["red_cards"] += away_red_cards
        self.team_stats[away_team]["second_yellow_cards"] += away_second_yellow_cards
        self.team_stats[away_team]["matches_played"] += 1

    def reorder_teams_by_new_order(self, sorted_teams, new_order):
        """
        Reorders the teams in sorted_teams based on the order provided in new_order,
        while keeping the overall ranking intact.
        
        Parameters:
        - sorted_teams: List of dictionaries, where each dictionary contains team data.
        - new_order: List of tuples with team names and their respective values (e.g., [('Team A', 0), ('Team B', 0)]).

        Returns:
        - List of dictionaries with teams ordered based on new_order while keeping the rest of the teams in their original order.
        """
        # Extraer solo los nombres de los equipos de la lista de tuplas
        new_order_teams = [team[0] for team in new_order]

        # Crear un diccionario con los equipos en sorted_teams para facilitar la búsqueda
        team_dict = {team['team']: team for team in sorted_teams}

        # Filtrar los equipos que están en el nuevo orden, preservando su estructura original
        reordered_block = [team_dict[team_name] for team_name in new_order_teams if team_name in team_dict]

        # Crear una nueva lista respetando la estructura de sorted_teams
        final_sorted_teams = []
        inserted = False

        for team in sorted_teams:
            if team['team'] in new_order_teams and not inserted:
                # Insertar el grupo reordenado en la posición correcta
                final_sorted_teams.extend(reordered_block)
                inserted = True  # Evitar insertar más de una vez
            elif team['team'] not in new_order_teams:
                # Mantener los equipos que no estaban en el nuevo orden
                final_sorted_teams.append(team)

        return final_sorted_teams

    def check_tiebreaker_type(self, sorted_teams, current_gameweek, index):
        # 1. Agrupar equipos por puntos
        points_groups = {}
        for team_stats in sorted_teams:
            points = team_stats["points"]
            if points not in points_groups:
                points_groups[points] = []
            points_groups[points].append(team_stats)  # Guardar en formato {equipo: estadísticas}

        # 2. Filtrar solo los grupos con más de un equipo (es decir, los empates)
        tied_groups = {points: teams for points, teams in points_groups.items() if len(teams) > 1}

        # 3. Reordenar los grupos de equipos empatados según criterios de desempate
        for points, teams_list in tied_groups.items():
            if len(teams_list) == 2:
                # Desempatar entre dos equipos
                sorted_teams = self.apply_tiebreaker_two_teams(sorted_teams, current_gameweek, index, teams_list, points)
            elif len(teams_list) >= 3:
                # Desempatar entre tres o más equipos
                sorted_teams = self.apply_tiebreaker_multiple_teams(sorted_teams, current_gameweek, index, teams_list, points)

        return sorted_teams

    def apply_tiebreaker_two_teams(self, sorted_teams, gameweek, match_id, teams_list, points):
        if len(teams_list) != 2:
            raise ValueError("apply_tiebreaker_two_teams must receive exactly two teams.")

        team1 = teams_list[0]["team"]
        team2 = teams_list[1]["team"]

        print(f"\nChecking tiebreaker for teams: {team1} vs {team2} (Points: {points}. Gameweek: {gameweek}. Match id: {match_id})")

        # Buscar partidos entre team1 y team2 hasta el partido actual (match_id)
        total_goals_team1 = 0
        total_goals_team2 = 0

        # Filtrar el CSV para obtener los partidos entre los dos equipos previos a este match_id
        for index, row in self.df.loc[:match_id-1].iterrows():  # Iterar desde el inicio hasta el id anterior
            home_team = row["home_team_name"]
            away_team = row["away_team_name"]
            home_goals = row["home_goals"]
            away_goals = row["away_goals"]

            # Verificar si el partido es entre los dos equipos
            if (home_team == team1 and away_team == team2) or (home_team == team2 and away_team == team1):
                print(f"Found match: {home_team} vs {away_team} (Home goals: {home_goals}, Away goals: {away_goals})")

                if home_team == team1:  # Si team1 es el equipo local
                    total_goals_team1 += home_goals
                    total_goals_team2 += away_goals
                else:  # Si team2 es el equipo local
                    total_goals_team1 += away_goals
                    total_goals_team2 += home_goals

        # Mostrar los goles totales marcados en los enfrentamientos directos
        print(f"Total goals in direct encounters: {team1}: {total_goals_team1}, {team2}: {total_goals_team2}")

        # Si no están empatados en goles en enfrentamientos directos, aplicar el desempate basado en los goles marcados en enfrentamientos directos
        if total_goals_team1 > total_goals_team2:
            print(f"{team1} has more goals in direct encounters. Moving {team1} up.")
            # Verificar si están en el orden correcto
            rank_team1 = sorted_teams.index(next(team for team in sorted_teams if team['team'] == team1))
            rank_team2 = sorted_teams.index(next(team for team in sorted_teams if team['team'] == team2))

            if rank_team1 > rank_team2:
                # Ajustar las posiciones
                sorted_teams[rank_team1], sorted_teams[rank_team2] = sorted_teams[rank_team2], sorted_teams[rank_team1]
        elif total_goals_team2 > total_goals_team1:
            print(f"{team2} has more goals in direct encounters. Moving {team2} up.")
            # Verificar si están en el orden correcto
            rank_team1 = sorted_teams.index(next(team for team in sorted_teams if team['team'] == team1))
            rank_team2 = sorted_teams.index(next(team for team in sorted_teams if team['team'] == team2))

            if rank_team2 > rank_team1:
                # Ajustar las posiciones
                sorted_teams[rank_team1], sorted_teams[rank_team2] = sorted_teams[rank_team2], sorted_teams[rank_team1]
        # Si los goles en enfrentamientos directos están empatados, aplicar el segundo criterio: diferencia de goles en todos los partidos
        else:
            total_goals_for_team1 = 0
            total_goals_against_team1 = 0
            total_goals_for_team2 = 0
            total_goals_against_team2 = 0

            # Filtrar el CSV para obtener los partidos de todos los equipos hasta el match_id
            for index, row in self.df.loc[:match_id-1].iterrows():  # Iterar desde el inicio hasta el id anterior
                home_team = row["home_team_name"]
                away_team = row["away_team_name"]
                home_goals = row["home_goals"]
                away_goals = row["away_goals"]

                # Para team1
                if home_team == team1:
                    total_goals_for_team1 += home_goals
                    total_goals_against_team1 += away_goals
                elif away_team == team1:
                    total_goals_for_team1 += away_goals
                    total_goals_against_team1 += home_goals

                # Para team2
                if home_team == team2:
                    total_goals_for_team2 += home_goals
                    total_goals_against_team2 += away_goals
                elif away_team == team2:
                    total_goals_for_team2 += away_goals
                    total_goals_against_team2 += home_goals

            # Calcular la diferencia de goles para cada equipo
            diff_goals_team1 = total_goals_for_team1 - total_goals_against_team1
            diff_goals_team2 = total_goals_for_team2 - total_goals_against_team2

            print(f"Goal difference in all encounters: {team1}: {diff_goals_team1}, {team2}: {diff_goals_team2}")

            # Aplicar el desempate basado en la diferencia de goles
            if diff_goals_team1 > diff_goals_team2:
                print(f"{team1} has a better goal difference in all encounters. Moving {team1} up.")
                # Verificar si están en el orden correcto
                rank_team1 = sorted_teams.index(next(team for team in sorted_teams if team['team'] == team1))
                rank_team2 = sorted_teams.index(next(team for team in sorted_teams if team['team'] == team2))

                if rank_team1 > rank_team2:
                    # Ajustar las posiciones
                    sorted_teams[rank_team1], sorted_teams[rank_team2] = sorted_teams[rank_team2], sorted_teams[rank_team1]
            elif diff_goals_team2 > diff_goals_team1:
                print(f"{team2} has a better goal difference in all encounters. Moving {team2} up.")
                # Verificar si están en el orden correcto
                rank_team1 = sorted_teams.index(next(team for team in sorted_teams if team['team'] == team1))
                rank_team2 = sorted_teams.index(next(team for team in sorted_teams if team['team'] == team2))

                if rank_team2 > rank_team1:
                    # Ajustar las posiciones
                    sorted_teams[rank_team1], sorted_teams[rank_team2] = sorted_teams[rank_team2], sorted_teams[rank_team1]
            else:
                # Buscar los diccionarios para team1 y team2
                team1_data = next(team for team in sorted_teams if team["team"] == team1)
                total_goals_team1 = team1_data["goals_for"]

                team2_data = next(team for team in sorted_teams if team["team"] == team2)
                total_goals_team2 = team2_data["goals_for"]
                
                if total_goals_team1 > total_goals_team2:
                    print(f"{team1} has scored more total goals. Keeping {team1} up.")
                elif total_goals_team1 < total_goals_team2:
                    print(f"{team2} has scored more total goals. Moving {team2} up.")
                    # Verificar si están en el orden correcto
                    rank_team1 = sorted_teams.index(next(team for team in sorted_teams if team['team'] == team1))
                    rank_team2 = sorted_teams.index(next(team for team in sorted_teams if team['team'] == team2))

                    if rank_team2 > rank_team1:
                        # Ajustar las posiciones
                        sorted_teams[rank_team1], sorted_teams[rank_team2] = sorted_teams[rank_team2], sorted_teams[rank_team1]
                else:
                    # Si los goles son iguales, comparar el fairplay
                    total_fairplay_team1 = team1_data["fairplay"]
                    total_fairplay_team2 = team2_data["fairplay"]

                    if total_fairplay_team1 < total_fairplay_team2:
                        print(f"{team1} has better fairplay (fewer penalties). Keeping {team1} up.")
                    elif total_fairplay_team1 > total_fairplay_team2:
                        print(f"{team2} has better fairplay (fewer penalties). Moving {team2} up.")
                        # Mover el equipo 2 hacia arriba (intercambiar posiciones)
                        rank_team1 = sorted_teams.index(next(team for team in sorted_teams if team['team'] == team1))
                        rank_team2 = sorted_teams.index(next(team for team in sorted_teams if team['team'] == team2))
                        
                        if rank_team2 > rank_team1:
                            # Ajustar las posiciones
                            sorted_teams[rank_team1], sorted_teams[rank_team2] = sorted_teams[rank_team2], sorted_teams[rank_team1]
                    else:
                        # Si aún están empatados, comparar por orden alfabético
                        winner = min(team1, team2)
                        print(f"{winner} has better alphabetical order. Moving {winner} up.")
                        if winner == team2:
                            rank_team1 = sorted_teams.index(next(team for team in sorted_teams if team['team'] == team1))
                            rank_team2 = sorted_teams.index(next(team for team in sorted_teams if team['team'] == team2))
        
                            # Ajustar las posiciones según el orden alfabético
                            if rank_team1 > rank_team2:
                                sorted_teams[rank_team1], sorted_teams[rank_team2] = sorted_teams[rank_team2], sorted_teams[rank_team1]

        return sorted_teams

    def apply_tiebreaker_multiple_teams(self, sorted_teams, gameweek, match_id, teams_list, points):
        """Apply tiebreaker logic between multiple teams, considering only their matches against each other."""

        print("\n0 - Ranking before tiebreaker:")
        print(", ".join([team_data["team"] for team_data in sorted_teams]))
        print(f"\nGroup with {points} points: {', '.join([team['team'] for team in teams_list])} at gameweek: {gameweek})")

        total_goals = {team["team"]: 0 for team in teams_list}
        total_points = {team["team"]: 0 for team in teams_list}

        for _, row in self.df.iterrows():
            if row["id"] <= match_id and (row["home_team_name"] in [team["team"] for team in teams_list]) and (row["away_team_name"] in [team["team"] for team in teams_list]):
                home_team = row["home_team_name"]
                away_team = row["away_team_name"]
                home_goals = row["home_goals"]
                away_goals = row["away_goals"]

                total_goals[home_team] += home_goals
                total_goals[away_team] += away_goals

                if home_goals > away_goals:
                    total_points[home_team] += 3
                elif home_goals < away_goals:
                    total_points[away_team] += 3
                else:
                    total_points[home_team] += 1
                    total_points[away_team] += 1

        points_sorted_teams = sorted(total_points.items(), key=lambda x: x[1], reverse=True)
        
        sorted_teams = self.reorder_teams_by_new_order(sorted_teams, points_sorted_teams)

        print("\n1 - Ranking after points between teams:")
        print(", ".join([team_data["team"] for team_data in sorted_teams]))

        points_groups = {}
        for team, pts in points_sorted_teams:
            if pts not in points_groups:
                points_groups[pts] = []
            points_groups[pts].append(team)

        filtered_points_groups = {pts: teams for pts, teams in points_groups.items() if len(teams) >= 2}

        for pts, teams in sorted(filtered_points_groups.items(), reverse=True):
            if len(teams) >= 2:
                for points, teams_in_group in points_groups.items():
                    print(f"\nGroup with {pts} points between teams: {', '.join(teams_in_group)} at gameweek: {gameweek}):")

                    if len(teams_in_group) >= 2:
                        total_goal_difference = {team: total_goals[team] - sum([total_goals[other_team] for other_team in teams_in_group if other_team != team]) for team in teams_in_group}

                        sorted_teams_by_goal_difference = sorted(total_goal_difference.items(), key=lambda x: x[1], reverse=True)
                        
                        sorted_teams = self.reorder_teams_by_new_order(sorted_teams, sorted_teams_by_goal_difference)

                        print("\n2 - Ranking after difference goals between teams:")
                        print(", ".join([team_data["team"] for team_data in sorted_teams]))

                        tied_teams = {}
                        current_tie_group = []
                        last_goal_diff = sorted_teams_by_goal_difference[0][1]

                        for team, goal_diff in sorted_teams_by_goal_difference:
                            if goal_diff == last_goal_diff:
                                current_tie_group.append(team)
                            else:
                                if len(current_tie_group) >= 2:
                                    tied_teams[len(current_tie_group)] = current_tie_group
                                current_tie_group = [team]
                                last_goal_diff = goal_diff

                        if len(current_tie_group) >= 2:
                            tied_teams[len(current_tie_group)] = current_tie_group

                        for tie_size, tie_group in tied_teams.items():
                            print(f"\nGroup with {goal_diff} goals difference between teams: {tie_group} at gameweek: {gameweek}):")
                            if tie_size >= 2:
                                goal_difference_total = {team: 0 for team in tie_group}

                                for _, row in self.df.iterrows():
                                    if row["id"] <= match_id and ((row["home_team_name"] in tie_group) or (row["away_team_name"] in tie_group)):
                                        home_team = row["home_team_name"]
                                        away_team = row["away_team_name"]
                                        home_goals = row["home_goals"]
                                        away_goals = row["away_goals"]

                                        if home_team in tie_group:
                                            goal_difference_total[home_team] += home_goals - away_goals
                                        if away_team in tie_group:
                                            goal_difference_total[away_team] += away_goals - home_goals

                                goal_diff_sorted_teams = sorted(goal_difference_total.items(), key=lambda x: x[1], reverse=True)

                                sorted_teams = self.reorder_teams_by_new_order(sorted_teams, goal_diff_sorted_teams)

                                print("\n3 - Ranking after difference goals global:")
                                print(", ".join([team_data["team"] for team_data in sorted_teams]))

                                tie_group = [team for team, _ in goal_diff_sorted_teams]

                                grouped_ties = {}
                                for team, goal_diff in goal_diff_sorted_teams:
                                    if goal_diff not in grouped_ties:
                                        grouped_ties[goal_diff] = []
                                    grouped_ties[goal_diff].append(team)

                                grouped_ties_with_multiple_teams = {goal_diff: teams for goal_diff, teams in grouped_ties.items() if len(teams) >= 2}

                                sorted_groups = sorted(grouped_ties_with_multiple_teams.items(), key=lambda x: x[0], reverse=True)

                                for goal_diff, teams in sorted_groups:
                                    print(f"\nGroup with {goal_diff} goals difference goals global: {teams} at gameweek: {gameweek}):")
                                    if len(teams) >= 2:
                                        goals_for_total = {team: 0 for team in teams}

                                        for _, row in self.df.iterrows():
                                            if row["gameweek"] <= gameweek and ((row["home_team_name"] in teams) or (row["away_team_name"] in teams)):
                                                home_team = row["home_team_name"]
                                                away_team = row["away_team_name"]
                                                home_goals = row["home_goals"]
                                                away_goals = row["away_goals"]

                                                if home_team in teams:
                                                    goals_for_total[home_team] += home_goals
                                                if away_team in teams:
                                                    goals_for_total[away_team] += away_goals

                                        global_diff_sorted_teams = sorted(goals_for_total.items(), key=lambda x: x[1], reverse=True)
                                        
                                        sorted_teams = self.reorder_teams_by_new_order(sorted_teams, global_diff_sorted_teams)
                                        
                                        print("\n4 - Ranking after goals global:")
                                        print(", ".join([team_data["team"] for team_data in sorted_teams]))

                                        grouped_ties = {}
                                        for team_name, goals_for in global_diff_sorted_teams:
                                            if goals_for not in grouped_ties:
                                                grouped_ties[goals_for] = []
                                            grouped_ties[goals_for].append((team_name, goals_for))  

                                        grouped_ties_list = [teams for teams in grouped_ties.values() if len(teams) >= 2]

                                        for group in grouped_ties_list:
                                            print(f"\nGroup: {[team[0] for team in group]} at gameweek: {gameweek})")

                                            if len(group) >= 2:
                                                teams_with_fairplay = []
                                                for team_name, _ in group:
                                                    team_data = next(team for team in sorted_teams if team["team"] == team_name)
                                                    fairplay = team_data["fairplay"]
                                                    teams_with_fairplay.append((team_name, fairplay))

                                                teams_with_fairplay_sorted = sorted(teams_with_fairplay, key=lambda x: x[1])

                                                sorted_teams_by_fairplay = [team[0] for team in teams_with_fairplay_sorted]

                                                group_sorted = sorted(group, key=lambda team: sorted_teams_by_fairplay.index(team[0]))

                                                sorted_teams = self.reorder_teams_by_new_order(sorted_teams, group_sorted)

                                                print("\n5 - Ranking after fairplay:")
                                                print(", ".join([team_data["team"] for team_data in sorted_teams]))

        return sorted_teams

    def calculate_fairplay(self, yellow_cards, red_cards, second_yellow_cards):
        """Calculates the fairplay score based on the card rules."""
        fairplay_score = yellow_cards + 3 * (red_cards - second_yellow_cards) + second_yellow_cards
        return fairplay_score

    def process_matches(self):
        # Inicializar variables
        current_gameweek = None
        gameweek_data = []  # Lista para almacenar los equipos y gameweek

        # Read the links and gameweeks from CSV files
        input_file = f"../database/{self.last_season}/matches_{self.last_season}.csv"
        output_file = f"../database/{self.last_season}/rankings_{self.last_season}.csv"
        self.df = pd.read_csv(input_file)

        # Borrar el archivo CSV si ya existe antes de empezar
        if os.path.exists(output_file):
            os.remove(output_file)

        # Iterar sobre las filas del CSV
        for index, row in self.df.iterrows():
            # Obtener el valor de la columna gameweek
            gameweek = row["gameweek"]

            # Si es el primer gameweek o el mismo que el anterior, agregar los nombres de los equipos a gameweek_data
            if current_gameweek is None or gameweek == current_gameweek:
                # Llamar a la función de actualización de estadísticas para los equipos
                self.update_team_stats(
                    home_team=row['home_team_name'],
                    away_team=row['away_team_name'],
                    home_goals=row["home_goals"],
                    away_goals=row["away_goals"],
                    home_yellow_cards=row["home_PlayersYellowCards"],
                    home_red_cards=row["home_PlayersRedCards"],
                    home_second_yellow_cards=row["home_PlayersSecondYellowCards"],
                    away_yellow_cards=row["away_PlayersYellowCards"],
                    away_red_cards=row["away_PlayersRedCards"],
                    away_second_yellow_cards=row["away_PlayersSecondYellowCards"]
                )

                # Obtener las estadísticas de los equipos después de la actualización
                home_stats = self.team_stats.get(row['home_team_name'], {})
                away_stats = self.team_stats.get(row['away_team_name'], {})

                # Añadir los datos de los equipos a gameweek_data
                gameweek_data.append({
                    'gameweek': gameweek,
                    'team': row['home_team_name'], 
                    'goals': row["home_goals"],
                    'yellow_cards': row["home_PlayersYellowCards"],
                    'red_cards': row["home_PlayersRedCards"],
                    'second_yellow_cards': row["home_PlayersSecondYellowCards"],
                    'date_of_match': row["date_of_match"],
                    "matches_played": home_stats.get("matches_played", 0),
                    "points": home_stats.get("points", 0),
                    "goals_for": home_stats.get("goals_for", 0),
                    "goals_against": home_stats.get("goals_against", 0),
                    "goal_difference": home_stats.get("goals_for", 0) - home_stats.get("goals_against", 0),
                    "yellow_cards": home_stats.get("yellow_cards", 0),
                    "red_cards": home_stats.get("red_cards", 0),
                    "second_yellow_cards": home_stats.get("second_yellow_cards", 0),
                    "fairplay": self.calculate_fairplay(home_stats.get("yellow_cards", 0), home_stats.get("red_cards", 0), home_stats.get("second_yellow_cards", 0)),
                })

                gameweek_data.append({
                    'gameweek': gameweek, 
                    'team': row['away_team_name'],
                    'goals': row["away_goals"],
                    'yellow_cards': row["away_PlayersYellowCards"],
                    'red_cards': row["away_PlayersRedCards"],
                    'second_yellow_cards': row["away_PlayersSecondYellowCards"],
                    'date_of_match': row["date_of_match"],
                    "matches_played": away_stats.get("matches_played", 0),
                    "points": away_stats.get("points", 0),
                    "goals_for": away_stats.get("goals_for", 0),
                    "goals_against": away_stats.get("goals_against", 0),
                    "goal_difference": away_stats.get("goals_for", 0) - away_stats.get("goals_against", 0),
                    "yellow_cards": away_stats.get("yellow_cards", 0),
                    "red_cards": away_stats.get("red_cards", 0),
                    "second_yellow_cards": away_stats.get("second_yellow_cards", 0),
                    "fairplay": self.calculate_fairplay(away_stats.get("yellow_cards", 0), away_stats.get("red_cards", 0), away_stats.get("second_yellow_cards", 0)),
                })

                current_gameweek = gameweek  # Actualizar el gameweek

            # Si encontramos un gameweek diferente, guardar los datos del gameweek anterior y reiniciar
            elif gameweek != current_gameweek:
                # Añadir el índice a cada fila de gameweek_data
                for entry in gameweek_data:
                    entry['index'] = index  # Asignar el valor del índice a cada fila de gameweek_data

                # Ordenar gameweek_data por puntos de mayor a menor
                gameweek_data.sort(key=lambda x: x['points'], reverse=True)

                # Ordenar y aplicar el desempate
                gameweek_data = self.check_tiebreaker_type(gameweek_data, current_gameweek, index)

                # Asignar el rank basado en la posición después de ordenar
                for rank, entry in enumerate(gameweek_data, start=1):
                    entry['rank'] = rank  # Asignar el ranko

                # Guardar los datos de este gameweek en el CSV
                gameweek_df = pd.DataFrame(gameweek_data)
                gameweek_df.to_csv(output_file, mode='a', index=False, header=not os.path.exists(output_file))  # Añadir al archivo CSV
                print(f"Datos guardados para gameweek {current_gameweek}")

                # Llamar a la función de actualización de estadísticas para los equipos
                self.update_team_stats(
                    home_team=row['home_team_name'],
                    away_team=row['away_team_name'],
                    home_goals=row["home_goals"],
                    away_goals=row["away_goals"],
                    home_yellow_cards=row["home_PlayersYellowCards"],
                    home_red_cards=row["home_PlayersRedCards"],
                    home_second_yellow_cards=row["home_PlayersSecondYellowCards"],
                    away_yellow_cards=row["away_PlayersYellowCards"],
                    away_red_cards=row["away_PlayersRedCards"],
                    away_second_yellow_cards=row["away_PlayersSecondYellowCards"]
                )

                # Obtener las estadísticas de los equipos después de la actualización
                home_stats = self.team_stats.get(row['home_team_name'], {})
                away_stats = self.team_stats.get(row['away_team_name'], {})

                # Limpiar la lista y actualizar el gameweek actual
                gameweek_data = [{
                    'gameweek': gameweek,
                    'team': row['home_team_name'], 
                    'goals': row["home_goals"],
                    'yellow_cards': row["home_PlayersYellowCards"],
                    'red_cards': row["home_PlayersRedCards"],
                    'second_yellow_cards': row["home_PlayersSecondYellowCards"],
                    'date_of_match': row["date_of_match"],
                    "matches_played": home_stats.get("matches_played", 0),
                    "points": home_stats.get("points", 0),
                    "goals_for": home_stats.get("goals_for", 0),
                    "goals_against": home_stats.get("goals_against", 0),
                    "goal_difference": home_stats.get("goals_for", 0) - home_stats.get("goals_against", 0),
                    "yellow_cards": home_stats.get("yellow_cards", 0),
                    "red_cards": home_stats.get("red_cards", 0),
                    "second_yellow_cards": home_stats.get("second_yellow_cards", 0),
                    "fairplay": self.calculate_fairplay(home_stats.get("yellow_cards", 0), home_stats.get("red_cards", 0), home_stats.get("second_yellow_cards", 0)),
                }, {
                    'gameweek': gameweek, 
                    'team': row['away_team_name'],
                    'goals': row["away_goals"],
                    'yellow_cards': row["away_PlayersYellowCards"],
                    'red_cards': row["away_PlayersRedCards"],
                    'second_yellow_cards': row["away_PlayersSecondYellowCards"],
                    'date_of_match': row["date_of_match"],
                    "matches_played": away_stats.get("matches_played", 0),
                    "points": away_stats.get("points", 0),
                    "goals_for": away_stats.get("goals_for", 0),
                    "goals_against": away_stats.get("goals_against", 0),
                    "goal_difference": away_stats.get("goals_for", 0) - away_stats.get("goals_against", 0),
                    "yellow_cards": away_stats.get("yellow_cards", 0),
                    "red_cards": away_stats.get("red_cards", 0),
                    "second_yellow_cards": away_stats.get("second_yellow_cards", 0),
                    "fairplay": self.calculate_fairplay(away_stats.get("yellow_cards", 0), away_stats.get("red_cards", 0), away_stats.get("second_yellow_cards", 0)),
                }]  # Empezar nueva lista con la fila actual

                current_gameweek = gameweek  # Actualizar el gameweek

        # Si quedan filas del último gameweek, guardarlas también
        if gameweek_data:
            # Añadir el índice a cada fila de gameweek_data
            for entry in gameweek_data:
                entry['index'] = index  # Asignar el valor del índice a cada fila de gameweek_data

            # Ordenar gameweek_data por puntos de mayor a menor
            gameweek_data.sort(key=lambda x: x['points'], reverse=True)

            # Ordenar y aplicar el desempate
            gameweek_data = self.check_tiebreaker_type(gameweek_data, current_gameweek, index)

            # Asignar el rank basado en la posición después de ordenar
            for rank, entry in enumerate(gameweek_data, start=1):
                entry['rank'] = rank  # Asignar el rank

            # Guardar las filas procesadas en un nuevo archivo CSV
            gameweek_df = pd.DataFrame(gameweek_data)
            gameweek_df.to_csv(output_file, mode='a', index=False, header=not os.path.exists(output_file))  # Añadir al archivo CSV

In [5]:
class MatchRankUpdater:
    def __init__(self, last_season):
        """
        Inicializa la clase con las rutas de los CSVs de los partidos y los rankings de los equipos.
        
        :param matches_csv: Ruta del archivo CSV con los partidos (home_team, away_team, date).
        :param teams_rank_csv: Ruta del archivo CSV con los equipos y rankings (team, date, rank, points).
        """
        self.last_season = last_season
        self.input_file = None
        self.output_file = None

    def _get_closest_stats(self, team, match_date):
        """
        Encuentra el ranking y los puntos más cercanos y anteriores a la fecha del partido para un equipo dado.
        """
        team_data = self.output_file[
            (self.output_file['team'] == team) & 
            (self.output_file['date_of_match'] < match_date)
        ]

        if not team_data.empty:
            closest_row = team_data.loc[team_data['date_of_match'].idxmax()]
            return closest_row['rank'], closest_row['points'], closest_row['goals_for'], closest_row['goals_against'], closest_row['goal_difference']
        
        return 0.0, 0.0, 0.0, 0.0, 0.0

    def update_and_save(self):
        """
        Actualiza el DataFrame de los partidos con los rankings y puntos de los equipos
        (home_team y away_team) y guarda el archivo CSV actualizado en el mismo archivo.
        """

        # Read the links and gameweeks from CSV files
        input_file = f"../database/{self.last_season}/matches_{self.last_season}.csv"
        output_file = f"../database/{self.last_season}/rankings_{self.last_season}.csv"
        self.input_file = pd.read_csv(input_file)
        self.output_file = pd.read_csv(output_file)

        self.input_file['date_of_match'] = pd.to_datetime(self.input_file['date_of_match'])
        self.output_file['date_of_match'] = pd.to_datetime(self.output_file['date_of_match'])
        
        home_team_ranks = []
        away_team_ranks = []
        home_team_points = []
        away_team_points = []
        home_team_goals_for = []
        away_team_goals_for = []
        home_team_goals_againsts = []
        away_team_goals_against = []
        home_team_goal_difference = []
        away_team_goal_difference = []

        # Iterar sobre las filas del DataFrame de los partidos
        for _, match_row in self.input_file.iterrows():
            match_date = match_row['date_of_match']  # Ya está en formato datetime
            home_team = match_row['home_team_name']
            away_team = match_row['away_team_name']
            
            # Obtener los rankings y puntos más cercanos para el home_team y away_team
            home_team_rank, home_team_pts, home_team_goal_for, home_team_goal_againsts, home_team_goals_difference = self._get_closest_stats(home_team, match_date)
            away_team_rank, away_team_pts, away_team_goal_for, away_team_goal_againsts, away_team_goals_difference = self._get_closest_stats(away_team, match_date)
            
            # Agregar los resultados a las listas
            home_team_ranks.append(home_team_rank)
            away_team_ranks.append(away_team_rank)
            home_team_points.append(home_team_pts)
            away_team_points.append(away_team_pts)
            home_team_goals_for.append(home_team_goal_for)
            away_team_goals_for.append(away_team_goal_for)
            home_team_goals_againsts.append(home_team_goal_againsts)
            away_team_goals_against.append(away_team_goal_againsts)
            home_team_goal_difference.append(home_team_goals_difference)
            away_team_goal_difference.append(away_team_goals_difference)

        # Añadir las columnas al DataFrame de partidos
        self.input_file['home_team_rank'] = home_team_ranks
        self.input_file['away_team_rank'] = away_team_ranks
        self.input_file['home_team_points'] = home_team_points
        self.input_file['away_team_points'] = away_team_points
        self.input_file['home_team_goals_for'] = home_team_goals_for
        self.input_file['away_team_goals_for'] = away_team_goals_for
        self.input_file['home_team_goals_againsts'] = home_team_goals_againsts
        self.input_file['away_team_goals_against'] = away_team_goals_against
        self.input_file['home_team_goals_difference'] = home_team_goal_difference
        self.input_file['away_team_goals_difference'] = away_team_goal_difference

        # Guardar el CSV actualizado en el mismo archivo
        self.input_file.to_csv(input_file, index=False)

## Obtaining odds data

In [6]:
class Odds_betting:
    def __init__(self, last_season):
        self.last_season = last_season
        self.betting_url = None
        self.matches_filename = None
        self.betting_filename = None
        self.betting_data = None
        self.match_data = None
        self.team_name_mapping = {
            "Leganes": "Leganes",
            "Alaves": "Alaves",
            "Valencia": "Valencia",
            "Las Palmas": "Las_Palmas",
            "Celta": "Celta_Vigo",
            "Sociedad": "Real_Sociedad",
            "Ath Madrid": "Atletico_Madrid",
            "Sevilla": "Sevilla",
            "Espanol": "Espanyol",
            "Ath Bilbao": "Athletic_Club",
            "Getafe": "Getafe",
            "Barcelona": "Barcelona",
            "Betis": "Real_Betis",
            "La Coruna": "Deportivo_La_Coruna",
            "Real Madrid": "Real_Madrid",
            "Levante": "Levante",
            "Villarreal": "Villarreal",
            "Malaga": "Malaga",
            "Eibar": "Eibar",
            "Girona": "Girona",
            "Granada": "Granada",
            "Vallecano": "Rayo_Vallecano",
            "Almeria": "Almeria",
            "Mallorca": "Mallorca",
            "Valladolid": "Valladolid"
        }

    def load_data(self):
        # Construir el código de temporada corto: "2024-2025" → "2425"
        short_season = self.last_season[2:4] + self.last_season[-2:]
        base_url = "https://www.football-data.co.uk/spainm.php"

        # Obtener HTML de la página
        response = requests.get(base_url)
        soup = BeautifulSoup(response.text, "html.parser")
        # Buscar la primera URL válida que apunte al CSV de La Liga
        self.betting_url = None
        for a in soup.find_all("a", href=True):
            href = a["href"]
            if f"mmz4281/{short_season}/SP1.csv" in href:
                self.betting_url = f"https://www.football-data.co.uk/{href}"
                break

        # Definir rutas locales
        self.matches_filename = f"../database/{self.last_season}/matches_{self.last_season}.csv"
        self.betting_filename = f"../database/{self.last_season}/betting_{self.last_season}.csv"

        # Descargar y guardar CSV
        if self.betting_url:
            os.makedirs(os.path.dirname(self.betting_filename), exist_ok=True)
            try:
                print(f"⬇️ Downloading betting data from {self.betting_url}")
                response = requests.get(self.betting_url)
                with open(self.betting_filename, "wb") as f:
                    f.write(response.content)
                print(f"✅ Betting CSV saved to: {self.betting_filename}")
            except Exception as e:
                print(f"❌ Failed to download or save CSV: {e}")
                self.betting_data = pd.DataFrame()
                return
        else:
            print("❌ No valid betting.csv URL found for this season.")
            self.betting_data = pd.DataFrame()
            return

        # Leer los CSVs
        try:
            self.betting_data = pd.read_csv(self.betting_filename)
            self.match_data = pd.read_csv(self.matches_filename)
        except Exception as e:
            print(f"❌ Error loading betting or ranking CSVs: {e}")
            self.betting_data = pd.DataFrame()
            self.match_data = pd.DataFrame()

    def rename_teams(self):
        if self.betting_data is not None:
            self.betting_data["HomeTeam"] = self.betting_data["HomeTeam"].replace(self.team_name_mapping)
            self.betting_data["AwayTeam"] = self.betting_data["AwayTeam"].replace(self.team_name_mapping)
        else:
            print("Error: betting data is None.")

    def extract_odds_columns(self):
        required_columns = [
            "Date", "HomeTeam", "AwayTeam",
            "B365H", "B365D", "B365A",
            "BWH", "BWD", "BWA",
            "BFH", "BFD", "BFA",
            "PSH", "PSD", "PSA",
            "WHH", "WHD", "WHA"
        ]
        available_columns = [col for col in required_columns if col in self.betting_data.columns]
        self.betting_data = self.betting_data[available_columns]

    def compute_avg_odds(self):
        def safe_mean(row, cols):
            values = [row[col] for col in cols if col in row and pd.notna(row[col])]
            return np.nanmean(values) if values else np.nan

        # Home odds
        self.betting_data["odd_home_avg"] = self.betting_data.apply(
            lambda row: row["BWH"] if pd.notna(row.get("BWH", np.nan))
            else safe_mean(row, ["B365H", "BFH", "PSH", "WHH"]),
            axis=1
        )

        # Draw odds
        self.betting_data["odd_draw_avg"] = self.betting_data.apply(
            lambda row: row["BWD"] if pd.notna(row.get("BWD", np.nan))
            else safe_mean(row, ["B365D", "BFD", "PSD", "WHD"]),
            axis=1
        )

        # Away odds
        self.betting_data["odd_away_avg"] = self.betting_data.apply(
            lambda row: row["BWA"] if pd.notna(row.get("BWA", np.nan))
            else safe_mean(row, ["B365A", "BFA", "PSA", "WHA"]),
            axis=1
        )
        
    def compute_probabilities(self):
        if self.betting_data["odd_home_avg"].isnull().any() or self.betting_data["odd_draw_avg"].isnull().any() or self.betting_data["odd_away_avg"].isnull().any():
            print("Warning: Some odds are missing or NaN.")

        self.betting_data["prob_home_avg"] = 1 / self.betting_data["odd_home_avg"]
        self.betting_data["prob_draw_avg"] = 1 / self.betting_data["odd_draw_avg"]
        self.betting_data["prob_away_avg"] = 1 / self.betting_data["odd_away_avg"]

        total_prob = self.betting_data["prob_home_avg"] + self.betting_data["prob_draw_avg"] + self.betting_data["prob_away_avg"]

        self.betting_data["prob_home_avg"] /= total_prob
        self.betting_data["prob_draw_avg"] /= total_prob
        self.betting_data["prob_away_avg"] /= total_prob

    def merge_with_match_data(self):
        if self.betting_data is not None and self.match_data is not None:
            # Columnas a insertar/actualizar
            prob_columns = ["prob_home_avg", "prob_draw_avg", "prob_away_avg"]

            # Si ya existen en match_data, las eliminamos
            for col in prob_columns:
                if col in self.match_data.columns:
                    self.match_data.drop(columns=col, inplace=True)

            # Merge con betting_data
            merged = self.match_data.merge(
                self.betting_data[["HomeTeam", "AwayTeam", "Date"] + prob_columns],
                how="left",
                left_on=["home_team_name", "away_team_name"],
                right_on=["HomeTeam", "AwayTeam"]
            )

            # Eliminar columnas auxiliares después del merge
            merged.drop(columns=["HomeTeam", "AwayTeam", "Date"], inplace=True)

            # Guardamos el resultado
            self.match_data = merged

        else:
            print("❌ Error: betting_data or match_data is None.")

    def save_updated_data(self):
        if self.match_data is not None:
            self.match_data.to_csv(self.matches_filename, index=False)
            print(f"Data with probabilities saved to {self.matches_filename}")
        else:
            print("Error: match_data is None. Cannot save data.")

    def process_odds(self):
        self.load_data()
        self.rename_teams()
        self.extract_odds_columns()
        self.compute_avg_odds()
        self.compute_probabilities()
        self.merge_with_match_data()
        self.save_updated_data()

## Working on stats

In [7]:
class MatchStatsCalculator:
    def __init__(self, seasons, long_strike, medium_strike, columns_to_work, existing_csv):
        self.df = None
        self.seasons = seasons
        self.long_strike = long_strike
        self.medium_strike = medium_strike
        self.columns_to_work = columns_to_work
        self.existing_csv = existing_csv
        self.new_df = None

    def get_dataframe(self):
        """
        Load, sort and concatenate match data from multiple CSV files.
        """
        df_list = []
        for season in self.seasons:
            file = f"../database/{season}/matches_{season}.csv"
            if os.path.exists(file):
                df_list.append(pd.read_csv(file))
            else:
                print(f"Warning: File {file} not found. Skipping...")

        if df_list:
            df = pd.concat(df_list, ignore_index=True)
            if df is not None and not df.empty:
                df = df.sort_values(by=["date_of_match", "hour_of_day"], ascending=True)
            else:
                raise ValueError("Data has not been loaded yet or is empty.")
        else:
            print("No valid files were loaded.")

        return df

    def calculate_avg_stat(self, column, strike, team1, current_date, location, team2=None, between_mode=False, opponent_mode=None, df=None):
        """
        Calculate the average of a given column based on mode:
        - None or False: standard stats for team1
        - "rival_avg": stats of team1's rivals (any role)
        - "rival_role_exact": stats of rivals of team1 when team1 plays in a specific role

        Returns:
            float: mean value or NaN
        """

        df["date_of_match"] = pd.to_datetime(df["date_of_match"], errors="coerce")

        if between_mode:
            if team2 is None:
                raise ValueError("You must provide team2 when using between_mode=True.")
            if location == "home":
                team_matches = df[
                    (df["date_of_match"] < current_date) & 
                    (df["home_team_name"] == team1) & 
                    (df["away_team_name"] == team2)
                ]
            elif location == "away":
                team_matches = df[
                    (df["date_of_match"] < current_date) & 
                    (df["away_team_name"] == team1) & 
                    (df["home_team_name"] == team2)
                ]
            else:
                team_matches = df[
                    (df["date_of_match"] < current_date) &
                    (((df["home_team_name"] == team1) & (df["away_team_name"] == team2)) |
                    ((df["away_team_name"] == team1) & (df["home_team_name"] == team2)))
                ]

        elif opponent_mode == "rival_avg":
            # partidos donde aparece team1 y tomamos el stat del rival (intercambiando home/away)
            team_matches = df[
                (df["date_of_match"] < current_date) &
                ((df["home_team_name"] == team1) | (df["away_team_name"] == team1))
            ]
            team_matches = team_matches.sort_values(by="date_of_match")
            while strike > 0:
                if len(team_matches) >= strike:
                    team_matches = team_matches.tail(strike)
                    break
                strike -= 1
            else:
                return np.nan

            values = []
            for _, row in team_matches.iterrows():
                if row["home_team_name"] == team1:
                    values.append(row[column.replace("home", "away")])
                elif row["away_team_name"] == team1:
                    values.append(row[column.replace("away", "home")])
            return np.mean(values)

        elif opponent_mode == "rival_role_exact":
            if location == "home":
                team_matches = df[
                    (df["date_of_match"] < current_date) &
                    (df["home_team_name"] == team1)
                ]
                target_column = column.replace("home", "away")  # visitante es el rival
            elif location == "away":
                team_matches = df[
                    (df["date_of_match"] < current_date) &
                    (df["away_team_name"] == team1)
                ]
                target_column = column.replace("away", "home")  # local es el rival
            else:
                raise ValueError("rival_role_exact only supports location='home' or 'away'")

            team_matches = team_matches.sort_values(by="date_of_match")
            while strike > 0:
                if len(team_matches) >= strike:
                    team_matches = team_matches.tail(strike)
                    break
                strike -= 1
            else:
                return np.nan

            return team_matches[target_column].mean()

        else:
            if location == "home":
                team_matches = df[
                    (df["date_of_match"] < current_date) & 
                    (df["home_team_name"] == team1)
                ]
            elif location == "away":
                team_matches = df[
                    (df["date_of_match"] < current_date) & 
                    (df["away_team_name"] == team1)
                ]
            else:
                team_matches = df[
                    (df["date_of_match"] < current_date) & 
                    ((df["home_team_name"] == team1) | (df["away_team_name"] == team1))
                ]

        team_matches = team_matches.sort_values(by="date_of_match")
        while strike > 0:
            if len(team_matches) >= strike:
                return team_matches.tail(strike)[column].mean()
            strike -= 1

        return np.nan
    
    def calculate_consecutive_stat(self, column, value, team1, current_date, location, team2, between_mode, operator, df=None):
        """
        Calculate the longest consecutive streak where 'column' meets a condition for a team.

        Args:
            column (str): Column representing the statistic to analyze.
            value (any): The specific value to compare (e.g., 1 for wins, 2 for goals, etc.).
            team1 (str): Name of the main team.
            current_date (str): Date of the current match.
            location (str): 'home', 'away', or 'both' to filter matches.
            team2 (str, optional): Second team to filter when `between_mode=True`.
            between_mode (bool): If True, considers only matches between `team1` and `team2`, but statistics are for `team1`.
            operator (str): Comparison operator ('=', '>', '<', '>=', '<=').

        Returns:
            int: Number of consecutive matches where 'column' meets the condition.
        """

        df["date_of_match"] = pd.to_datetime(df["date_of_match"], errors="coerce")

        if location == "home":
            team_matches = df[
                (df["date_of_match"] < current_date) & 
                (df["home_team_name"] == team1)
            ]
        elif location == "away":
            team_matches = df[
                (df["date_of_match"] < current_date) & 
                (df["away_team_name"] == team1)
            ]
        else:
            team_matches = df[
                (df["date_of_match"] < current_date) & 
                ((df["home_team_name"] == team1) | (df["away_team_name"] == team1))
            ]

        if between_mode:
            if team2 is None:
                raise ValueError("You must provide team2 when using between_mode=True.")
            team_matches = team_matches[
                ((team_matches["home_team_name"] == team1) & (team_matches["away_team_name"] == team2)) |
                ((team_matches["home_team_name"] == team2) & (team_matches["away_team_name"] == team1))
            ]

        team_matches = team_matches.sort_values(by="date_of_match", ascending=False)

        if len(team_matches) == 0:
            return 0

        streak = 0
        for _, row in team_matches.iterrows():
            row_value = row[column]

            if (
                (operator == "eq" and row_value == value) or
                (operator == "gt" and row_value > value) or
                (operator == "lt" and row_value < value) or
                (operator == "ge" and row_value >= value) or
                (operator == "le" and row_value <= value)
            ):
                streak += 1
            else:
                break

        return streak

    def update_columns(self, choice_2=None, start_index=0):
        print("Collecting data...")
        df = self.get_dataframe()
        count = len(df)

        home_away_location = ["home", "away"]
        existing_csv = pd.read_csv(self.existing_csv)

        long_strikes = list(range(1, self.long_strike + 1))
        medium_strikes = list(range(1, self.medium_strike + 1))
        new_values = {}
        columns_to_add = {}

        if choice_2 is not None:
            df_to_process = df.tail(choice_2).reset_index(drop=True)
            indices_to_process = list(range(start_index, start_index + len(df_to_process)))
        else:
            df_to_process = df
            indices_to_process = list(range(len(df_to_process)))

        for offset, (idx, row) in enumerate(zip(indices_to_process, df_to_process.itertuples(index=False))):
            current_date = getattr(row, "date_of_match")
            print(f"Working on row {idx + 1} of {count}...")

            for home_away in home_away_location:
                if home_away == "home":
                    team = getattr(row, "home_team_name")
                    vs_team = getattr(row, "away_team_name")
                    first_location = "home"
                    other_home_away = "away"
                else:
                    team = getattr(row, "away_team_name")
                    vs_team = getattr(row, "home_team_name")
                    first_location = "away"
                    other_home_away = "home"

                for column, (values, operators) in self.columns_to_work.items():
                    for strike in long_strikes:
                        for suffix, loc in [
                            (f"_{home_away}", "both"),
                            (f"_{home_away}_at_{home_away}", first_location)
                        ]:
                            col_name = f"avg_{home_away}_{column}_last_{strike}{suffix}"
                            if col_name not in df.columns:
                                columns_to_add[col_name] = np.nan
                            if choice_2 is None and col_name in existing_csv.columns:
                                continue
                            value = self.calculate_avg_stat(f"{home_away}_{column}", strike, team, current_date, location=loc, df=df)
                            new_values.setdefault(col_name, {})[idx] = value

                        for suffix, loc in [
                            (f"_rivals_of_{other_home_away}", "both"),
                            (f"_rivals_of_{other_home_away}_at_{home_away}", first_location)
                        ]:
                            col_name = f"avg_{home_away}_{column}_last_{strike}{suffix}"
                            if col_name not in df.columns:
                                columns_to_add[col_name] = np.nan
                            if choice_2 is None and col_name in existing_csv.columns:
                                continue
                            opponent_mode = "rival_avg" if "rivals_of_" in suffix and "_at_" not in suffix else "rival_role_exact"
                            value = self.calculate_avg_stat(f"{home_away}_{column}", strike, vs_team, current_date, location=loc, opponent_mode=opponent_mode, df=df)
                            new_values.setdefault(col_name, {})[idx] = value

                    for strike in medium_strikes:
                        for suffix, loc in [
                            (f"_{home_away}_vs_{other_home_away}", "both"),
                            (f"_{home_away}_vs_{other_home_away}_at_{home_away}", first_location)
                        ]:
                            col_name = f"avg_{home_away}_{column}_last_{strike}{suffix}"
                            if col_name not in df.columns:
                                columns_to_add[col_name] = np.nan
                            if choice_2 is None and col_name in existing_csv.columns:
                                continue
                            value = self.calculate_avg_stat(f"{home_away}_{column}", strike, team, current_date, location=loc, team2=vs_team, between_mode=True, df=df)
                            new_values.setdefault(col_name, {})[idx] = value

                    for value, operator in zip(values, operators):
                        base = f"consecutive_{home_away}_{column}_{operator}_{value}"
                        variants = [
                            (f"_{home_away}", team, "both", None, False),
                            (f"_{home_away}_at_{home_away}", team, first_location, None, False),
                            (f"_{home_away}_vs_{other_home_away}", team, "both", vs_team, True),
                            (f"_{home_away}_vs_{other_home_away}_at_{home_away}", team, "home", vs_team, True)
                        ]
                        for suffix, team_used, loc, team2, between in variants:
                            col_name = f"{base}{suffix}"
                            if col_name not in df.columns:
                                columns_to_add[col_name] = np.nan
                            if choice_2 is None and col_name in existing_csv.columns:
                                continue
                            val = self.calculate_consecutive_stat(f"{home_away}_{column}", value, team_used, current_date, location=loc, team2=team2, between_mode=between, operator=operator, df=df)
                            new_values.setdefault(col_name, {})[idx] = val

        if columns_to_add:
            df = pd.concat([df, pd.DataFrame(columns_to_add, index=df.index)], axis=1)

        new_values_df = pd.DataFrame(new_values)
        new_values_df.index = new_values_df.index.astype(int)

        missing_cols = [col for col in new_values_df.columns if col not in existing_csv.columns]
        if missing_cols:
            missing_df = pd.DataFrame(np.nan, index=existing_csv.index, columns=missing_cols)
            existing_csv = pd.concat([existing_csv, missing_df], axis=1)

        for col in new_values_df.columns:
            for idx, value in new_values_df[col].items():
                if idx < len(existing_csv):
                    existing_csv.at[idx, col] = value

        existing_csv.to_csv(self.existing_csv, index=False)
        self.new_df = new_values_df

    def update_or_add_columns(self):
        choice_1 = input("What do you want to do? (1: Add new columns or 2: Update with the latest matches): ").strip()

        if choice_1 == "1":
            self.update_columns()

            existing_df = pd.read_csv(self.existing_csv)

            print(f"New columns successfully added to {self.existing_csv}")

        elif choice_1 == "2":
            choice_2 = int(input("How many matches do you want to add? ").strip())

            existing_df = pd.read_csv(self.existing_csv)
            original_length = len(existing_df)

            last_season = self.seasons[-1]

            Match_data_extractor = Match_data(last_season, choice_2)
            Match_data_extractor.run()

            Players_data_extractor = Players_data(last_season, choice_2)
            Players_data_extractor.run()

            ranking = FootballRanking(last_season)
            ranking.process_matches()
            updater = MatchRankUpdater(last_season)
            updater.update_and_save()

            odds_processor = Odds_betting(last_season)
            odds_processor.process_odds()

            filename = f"../database/{last_season}/matches_{last_season}.csv"
            match_df = pd.read_csv(filename)

            columns_to_update = [
                "gameweek", "id", "date_of_match", "hour_of_the_match", "day_of_week", "day_of_year", "hour_of_day",
                "home_team_name", "away_team_name", "home_trainer", "away_trainer", "stadium", 
                "home_goals", "away_goals", "result", "home_team_rank", "away_team_rank", "home_team_points", "away_team_points",
                "home_team_goals_for", "away_team_goals_for", "home_team_goals_againsts", "away_team_goals_against",
                "home_team_goals_difference", "away_team_goals_difference", "referee", "var", "prob_home_avg",
                "prob_draw_avg", "prob_away_avg"
            ]

            columns_valid = [col for col in columns_to_update if col in match_df.columns and col in existing_df.columns]

            matches_to_add = match_df[columns_valid].tail(choice_2).copy()

            combined_df = pd.concat([existing_df, matches_to_add], ignore_index=True)

            combined_df.to_csv(self.existing_csv, index=False)

            self.update_columns(choice_2, start_index=original_length)

            print(f"New columns and matches successfully added to {self.existing_csv}")

        else:
            print("Invalid option. Please choose 1, 2 or 3.")

# MODEL TESTING

In [8]:
class ModelPipeline:
    def __init__(self, existing_csv, categorical_features, circular_max_values, ranking_features, target, random_state, seasons, long_strike, medium_strike, features_to_work):
        self.existing_csv = existing_csv
        self.categorical_features = categorical_features
        self.circular_max_values = circular_max_values
        self.ranking_features = ranking_features
        self.target = target
        self.random_state = random_state
        self.best_params = None
        self.best_split = None
        self.best_features = None
        self.best_f1_weighted = None
        self.precision_weighted_cv = None
        self.recall_weighted_cv = None
        self.best_description = None
        self.description = None
        self.number_experiment = None
        self.seasons = seasons
        self.long_strike = long_strike
        self.medium_strike = medium_strike
        self.features_to_work = features_to_work
        self.df = None

    def _load_data(self):
        data = pd.read_csv(self.existing_csv)

        categorical_features = self.categorical_features

        self.label_encoders = {}  # <--- guardar encoders para luego usarlos en predicción
        for col in categorical_features:
            if col in data.columns:
                le = LabelEncoder()
                data[col] = le.fit_transform(data[col])
                self.label_encoders[col] = le  # <--- guardar el encoder

        circular_max_values = self.circular_max_values
        for col, max_value in circular_max_values.items():
            if col in data.columns:
                data[f"{col}_sin"] = np.sin(2 * np.pi * data[col] / max_value)
                data[f"{col}_cos"] = np.cos(2 * np.pi * data[col] / max_value)
                data.drop(columns=[col], inplace=True)

        return data

    def fetch_latest_mlflow_run(self):

        def safe_cast(value):
            v = value.strip() if isinstance(value, str) else value
            if isinstance(v, str):
                if v.lower() == "true":
                    return True
                elif v.lower() == "false":
                    return False
                elif v.lower() == "none":
                    return None
                try:
                    return float(v) if '.' in v else int(v)
                except ValueError:
                    return v
            return v
        
        experiment_name = "BetPredictions"
        tracking_uri = os.path.abspath("mlruns")
        mlflow.set_tracking_uri(f"file://{tracking_uri}")
        client = mlflow.tracking.MlflowClient()

        experiment = next(
            (exp for exp in mlflow.search_experiments() if exp.name.strip().lower() == experiment_name.strip().lower()),
            None
        )

        if experiment is None:
            print(f"No experiment found with name: {experiment_name}")
            return

        # Obtener los últimos 50 runs (puedes aumentar si hace falta)
        runs = client.search_runs(
            experiment.experiment_id,
            order_by=["start_time desc"],
            max_results=50
        )

        # Filtrar solo los runs que tengan un runName tipo XGBoost_{numero}
        xgb_runs = []
        for run in runs:
            run_name = run.data.tags.get("mlflow.runName", "")
            match = re.match(r"XGBoost_(\d+)", run_name)
            if match:
                run_number = int(match.group(1))
                xgb_runs.append((run_number, run))

        if not xgb_runs:
            print("❌ No XGBoost_n runs found.")
            return

        # Seleccionar el run con el número más alto
        latest_number, latest_run = max(xgb_runs, key=lambda x: x[0])
        self.number_experiment = latest_number

        # Casteo de parámetros
        raw_params = latest_run.data.params.copy()
        raw_params["n_splits"] = raw_params.pop("Best split number", 3)

        self.best_params = {k: safe_cast(v) for k, v in raw_params.items()}
        self.best_split = self.best_params.pop("n_splits", 3)
        self.best_features = latest_run.data.params.get("Features", "").split(',')

        self.f1_weighted_cv = latest_run.data.metrics.get("f1_weighted_cv", 0.0)
        self.precision_weighted_cv = latest_run.data.metrics.get("precision_weighted_cv", 0.0)
        self.recall_weighted_cv = latest_run.data.metrics.get("recall_weighted_cv", 0.0)

        self.best_description = latest_run.data.tags.get('mlflow.note.content', 'No description available')

        # (Opcional) Abrir la UI de MLflow
        try:
            subprocess.Popen(["mlflow", "ui"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            time.sleep(5)
            mlflow_url = f"http://127.0.0.1:5000/#/experiments/{experiment.experiment_id}"
            subprocess.Popen(["xdg-open", mlflow_url])
        except Exception as e:
            print(f"⚠️ Could not open MLflow UI: {e}")

    def run_optuna_precision(self, data, features, target, n_trials=500, forced_params=None, forced_split=None):
        print("\n🔍 Starting Optuna optimization for PRECISION...")

        X = data[features]
        y = data[target]

        optuna.logging.set_verbosity(optuna.logging.ERROR)
        sampler = TPESampler(n_startup_trials=n_trials/10, seed=42)
        study = optuna.create_study(direction="maximize", sampler=sampler)

        best_precision_so_far = 0.0
        best_params = None
        best_n_splits = None
        best_splits = None

        def objective(trial):
            nonlocal best_precision_so_far, best_params, best_n_splits, best_splits

            # Suggested hyperparameters
            params = {
                "max_depth": trial.suggest_int("max_depth", 3, 6),
                "learning_rate": trial.suggest_float("learning_rate", 0.005, 0.02, log=True),
                "subsample": trial.suggest_float("subsample", 0.6, 0.95),
                "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
                "gamma": trial.suggest_float("gamma", 0, 5),
                "reg_alpha": trial.suggest_float("reg_alpha", 0, 10),
                "reg_lambda": trial.suggest_float("reg_lambda", 0, 10),
                "min_child_weight": trial.suggest_int("min_child_weight", 1, 8),
                "n_estimators": trial.suggest_int("n_estimators", 200, 600),
                "booster": "gbtree",
                "objective": "multi:softprob",
                "num_class": 3,
                "random_state": self.random_state,
                "tree_method": "hist",
                "device": "cuda",
                "max_bin": 256
            }

            n_splits = trial.suggest_int("n_splits", 3, 5)
            tscv = TimeSeriesSplit(n_splits=n_splits)
            precision_scores = []
            recall_scores = []

            for train_idx, test_idx in tscv.split(X):
                X_train, X_test = X.iloc[train_idx].copy(), X.iloc[test_idx].copy()
                y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

                # Encode categorical features if any
                for col in X_train.select_dtypes(include="category").columns:
                    X_train[col] = X_train[col].cat.codes
                for col in X_test.select_dtypes(include="category").columns:
                    X_test[col] = X_test[col].cat.codes

                clf = XGBClassifier(**params)
                clf.fit(X_train, y_train)

                y_pred = clf.predict(X_test.fillna(0))
                report = classification_report(y_test, y_pred, labels=[0, 1, 2], output_dict=True, zero_division=0)
                precision_scores.append(report["weighted avg"]["precision"])
                recall_scores.append(report["weighted avg"]["recall"])

            avg_precision = np.mean(precision_scores)
            avg_recall = np.mean(recall_scores)

            trial.set_user_attr("avg_precision", avg_precision)
            trial.set_user_attr("avg_recall", avg_recall)

            if avg_precision > best_precision_so_far:
                best_precision_so_far = avg_precision
                best_params = params.copy()
                best_n_splits = n_splits
                best_splits = list(tscv.split(X))

                print(f"✅ Trial {trial.number} → precision: {avg_precision:.4f}, recall: {avg_recall:.4f}")

            return avg_precision

        if forced_params:
            trial_config = forced_params.copy()
            if forced_split is not None:
                trial_config["n_splits"] = forced_split
            study.enqueue_trial(trial_config)

        study.optimize(objective, n_trials=n_trials)

        avg_precision = study.best_trial.user_attrs.get("avg_precision", 0.0)
        avg_recall = study.best_trial.user_attrs.get("avg_recall", 0.0)

        print(f"\n🔵 Best precision trial finished with:")
        print(f"   → Params        : {best_params}")
        print(f"   → Splits        : {best_n_splits}")
        print(f"   → Avg precision : {avg_precision:.4f}")
        print(f"   → Avg recall    : {avg_recall:.4f}")

        return best_params, best_n_splits, best_splits, avg_precision, avg_recall

    def feature_selection(self, params, original_base_features, enriched_features, data, target):
        # 0. Remove duplicates between base and enriched features
        base_features = original_base_features.copy()
        enriched_features = enriched_features.copy()

        X_enriched = data[enriched_features]
        y_target_global = data[target]

        print("🔵 Training XGBoost to get feature importances (GPU forced)...")

        # Copy Optuna params and force GPU-related parameters
        xgb_params = params.copy()
        xgb_params.update({
            "device": "cuda",
            "tree_method": "hist",
            "enable_categorical": True,
            "random_state": self.random_state,
            "n_jobs": 1
        })

        try:
            xgb_model = XGBClassifier(**xgb_params)
            xgb_model.fit(X_enriched.fillna(0), y_target_global)

        except Exception as e:
            if "cudaErrorMemoryAllocation" in str(e):
                print("⚠️ GPU memory exceeded. Switching to CPU fallback...")

                xgb_params.pop("device", None)
                xgb_params["n_jobs"] = -1
                xgb_model = XGBClassifier(**xgb_params)
                xgb_model.fit(X_enriched.fillna(0), y_target_global)
            else:
                raise

        importances_global = pd.Series(xgb_model.feature_importances_, index=X_enriched.columns)
        top_50_features = importances_global.nlargest(250).index.tolist()
        importances = importances_global

        # 2. Extract top 10 home/away features per group
        top_features = []

        for group in self.features_to_work:
            home_group_features = [f for f in importances.index if f"home_{group}" in f]
            away_group_features = [f for f in importances.index if f"away_{group}" in f]

            if home_group_features:
                top_home = importances[home_group_features].sort_values(ascending=False).head(1).index[0]
                top_features.append(top_home)

            if away_group_features:
                top_away = importances[away_group_features].sort_values(ascending=False).head(1).index[0]
                top_features.append(top_away)

        # 3. Final list of features, ensuring uniqueness
        circular_features = [
            'hour_of_day_sin', 'hour_of_day_cos',
            'day_of_week_sin', 'day_of_week_cos',
            'gameweek_sin', 'gameweek_cos',
            'day_of_year_sin', 'day_of_year_cos'
        ]

        for col in self.categorical_features:
            if col in data.columns:
                data[col] = data[col].astype("category")

        proposed_features = (
            top_features
            + top_50_features
            + base_features
            + self.categorical_features
            + self.ranking_features
            + circular_features
            + self.best_features
        )

        # Verifica qué columnas están presentes
        train_features = [f for f in proposed_features if f in data.columns]
        train_features = list(dict.fromkeys(train_features))

        missing = [f for f in proposed_features if f not in data.columns]
        if missing:
            print(f"⚠️ Ignored {len(missing)} missing features: {missing}")

        print(f"✅ Selected {len(train_features)} total features after group-based XGBoost importance selection.")
        return train_features

    def iterative_feature_selection(
        self,
        original_base_features: list,
        train_features: list,
        test_hyperparams: dict,
        splits: list[tuple[np.ndarray, np.ndarray]],
        random_state: int,
        data: pd.DataFrame,
        target: str
    ) -> tuple[list, float, float]:
        """
        Iteratively selects features that improve weighted precision using XGBoost and fixed splits.

        Parameters:
        - original_base_features: List of mandatory features.
        - train_features: Candidate features to evaluate.
        - test_hyperparams: Hyperparameters for XGBClassifier.
        - splits: Predefined list of (train_index, test_index) tuples.
        - random_state: Seed.
        - data: DataFrame with all data.
        - target: Target column name.

        Returns:
        - selected_features: Final selected features.
        - best_precision: Best weighted precision achieved.
        - best_recall: Corresponding recall.
        """

        train_features = [f for f in train_features if f not in original_base_features]
        all_features = list(dict.fromkeys(original_base_features + train_features))

        X = data[all_features].copy()
        X = X.loc[:, ~X.columns.duplicated()]
        y = data[target]

        base_features = original_base_features.copy()
        current_best_precision_score = 0.0
        current_best_recall_score = 0.0

        while train_features:
            improved = False
            best_new_feature = None

            for feature in tqdm(train_features, desc="Evaluating features"):
                current_features = base_features + [feature]
                global_metrics_list = []

                for train_index, test_index in splits:
                    X_train = X.iloc[train_index][current_features].copy()
                    X_test = X.iloc[test_index][current_features].copy()
                    y_train = y.iloc[train_index].copy()
                    y_test = y.iloc[test_index].copy()

                    for col in X_train.select_dtypes(include="category").columns:
                        X_train[col] = X_train[col].cat.codes
                    for col in X_test.select_dtypes(include="category").columns:
                        X_test[col] = X_test[col].cat.codes

                    model = XGBClassifier(**test_hyperparams)
                    model.fit(X_train, y_train)
                    y_pred = model.predict(X_test.fillna(0))

                    report = classification_report(
                        y_test, y_pred,
                        labels=[0.0, 1.0, 2.0],
                        output_dict=True,
                        zero_division=0
                    )

                    global_metrics_list.append({
                        "f1_macro": report.get("macro avg", {}).get("f1-score", 0.0),
                        "f1_weighted": report.get("weighted avg", {}).get("f1-score", 0.0),
                        "precision_macro": report.get("macro avg", {}).get("precision", 0.0),
                        "recall_macro": report.get("macro avg", {}).get("recall", 0.0),
                        "precision_weighted": report.get("weighted avg", {}).get("precision", 0.0),
                        "recall_weighted": report.get("weighted avg", {}).get("recall", 0.0)
                    })

                avg_metrics = pd.DataFrame(global_metrics_list).mean().to_dict()
                precision_w = avg_metrics.get("precision_weighted", 0.0)
                recall_w = avg_metrics.get("recall_weighted", 0.0)

                # Check if this feature improves precision
                if precision_w > current_best_precision_score:
                    current_best_precision_score = precision_w
                    current_best_recall_score = recall_w
                    best_new_feature = feature
                    improved = True

            if improved and best_new_feature:
                base_features.append(best_new_feature)
                train_features.remove(best_new_feature)
                print(f"✅ Added feature: {best_new_feature}")
                print(f"   → Precision: {current_best_precision_score:.4f}, Recall: {current_best_recall_score:.4f}")
                print(f"   → Current feature set: {base_features}")
            else:
                print(f"❌ No improvement. Final feature set → Precision: {current_best_precision_score:.4f}, Recall: {current_best_recall_score:.4f}")
                break

        return base_features, current_best_precision_score, current_best_recall_score

    def evaluate_and_log_model_if_improved(
        self,
        data: pd.DataFrame,
        target: str,
        best_features: list,
        test_hyperparams: dict,
        best_split_number: int,
        best_splits: list[tuple[np.ndarray, np.ndarray]],  # fixed CV splits
        iteration_precision: float,
        iteration_recall: float,
        best_precision: float,
        best_recall: float
    ) -> tuple[bool, float, float]:
        """
        Evalúa un modelo y lo registra en MLflow si mejora según los criterios definidos.
        """

        #condition1 = iteration_precision >= best_precision and iteration_recall > 0.95 * best_recall
        #condition2 = iteration_precision > best_precision and iteration_recall >= best_recall

        condition = iteration_precision > best_precision

        if condition:
            X = data[best_features]
            y = data[target]

            fold_class_metrics = []
            fold_global_metrics = []

            for train_index, test_index in best_splits:
                X_train = X.iloc[train_index].copy()
                X_test = X.iloc[test_index].copy()
                y_train = y.iloc[train_index]
                y_test = y.iloc[test_index]

                # Convert categorical columns to int codes to avoid GPU/QuantileDMatrix issues
                for col in X_train.select_dtypes(include="category").columns:
                    X_train[col] = X_train[col].cat.codes
                for col in X_test.select_dtypes(include="category").columns:
                    X_test[col] = X_test[col].cat.codes

                model = XGBClassifier(**test_hyperparams)
                model.fit(X_train, y_train)

                y_pred = model.predict(X_test.fillna(0))

                report = classification_report(
                    y_test, y_pred,
                    labels=[0.0, 1.0, 2.0],
                    output_dict=True,
                    zero_division=0
                )

                fold_global_metrics.append({
                    "f1_macro": report.get("macro avg", {}).get("f1-score", 0.0),
                    "f1_weighted": report.get("weighted avg", {}).get("f1-score", 0.0),
                    "precision_macro": report.get("macro avg", {}).get("precision", 0.0),
                    "recall_macro": report.get("macro avg", {}).get("recall", 0.0),
                    "precision_weighted": report.get("weighted avg", {}).get("precision", 0.0),
                    "recall_weighted": report.get("weighted avg", {}).get("recall", 0.0)
                })

                fold_class_metrics.append({
                    label: report.get(label, {
                        "precision": 0.0,
                        "recall": 0.0,
                        "f1-score": 0.0,
                        "support": 0.0
                    }) for label in ["0.0", "1.0", "2.0"]
                })

            # Aggregate metrics
            metrics_df = pd.DataFrame(fold_global_metrics)
            avg_metrics = metrics_df.mean().to_dict()

            per_class_avg = {}
            for label in ["0.0", "1.0", "2.0"]:
                rows = [fold[label] for fold in fold_class_metrics if label in fold]
                if rows:
                    per_class_avg[label] = pd.DataFrame(rows).mean().to_dict()
                else:
                    print(f"⚠️ No data for class '{label}' across folds.")
                    per_class_avg[label] = {
                        "precision": 0.0,
                        "recall": 0.0,
                        "f1-score": 0.0,
                        "support": 0.0
                    }

            # Save results
            self.f1_macro_cv = avg_metrics.get("f1_macro", 0.0)
            self.f1_weighted_cv = avg_metrics.get("f1_weighted", 0.0)
            self.precision_macro_cv = avg_metrics.get("precision_macro", 0.0)
            self.recall_macro_cv = avg_metrics.get("recall_macro", 0.0)
            self.precision_weighted_cv = avg_metrics.get("precision_weighted", 0.0)
            self.recall_weighted_cv = avg_metrics.get("recall_weighted", 0.0)

            self.best_params = test_hyperparams
            self.best_split = best_split_number
            self.best_features = best_features
            self.cv_avg_class_metrics = per_class_avg
            self.X_final = X
            self.y_final = y

            self.log_to_mlflow()

            print(f"\n✅ Model accepted and logged to MLflow.")
        else:
            print(f"\n🛑 Model discarded. Working...")

    def reemplazar_simbolos(self, nombre):
        nombre = nombre.replace(">=", "ge")
        nombre = nombre.replace("<=", "le")
        nombre = nombre.replace(">", "gt")
        nombre = nombre.replace("<", "lt")
        nombre = nombre.replace("=", "eq")
        return nombre

    def train(self):

        logging.getLogger("mlflow").setLevel(logging.ERROR)

        data = self._load_data()
        self.fetch_latest_mlflow_run()

        for col in data.select_dtypes(include="category").columns:
            data[col] = data[col].cat.codes

        target = self.target
        original_base_features = ["home_team_name", "away_team_name"]
        enriched_features = [col for col in data.columns if col not in [target, "date_of_match"]]

        # Apply symbol replacements
        self.best_features = [self.reemplazar_simbolos(f) for f in self.best_features]

        best_params = self.best_params.copy()
        best_split = self.best_split
        best_features = self.best_features
        best_precision = self.precision_weighted_cv
        best_recall = self.recall_weighted_cv

        best_split = int(best_split)
        best_params.pop("Features", None)

        print(f"Best model until now. Params: {best_params}. Splits: {best_split}. Best features: {best_features}.")
        print(f"Precision score: {best_precision}.")
        print(f"Recall score: {best_recall}.")

        train_features=self.feature_selection(
            best_params, 
            original_base_features, 
            enriched_features, 
            data, 
            target
        )

        best_params, best_n_splits, best_split, optuna_precision, optuna_recall = self.run_optuna_precision(
            data, 
            train_features, 
            target,
            n_trials=500,
            #forced_params=best_params,
            #forced_split=best_split
        )

        self.evaluate_and_log_model_if_improved(
            data=data,
            target="result",
            best_features=train_features,
            test_hyperparams=best_params,
            best_split_number=best_n_splits,
            best_splits=best_split,
            iteration_precision=optuna_precision,
            iteration_recall=optuna_recall,
            best_precision=self.precision_weighted_cv,
            best_recall=self.recall_weighted_cv
        )

        while True:

            iteration_features, iteration_precision, iteration_recall = self.iterative_feature_selection(
                original_base_features=original_base_features,
                train_features=train_features,
                test_hyperparams=best_params,
                splits=best_split,
                random_state=self.random_state,
                data=data,
                target=target
            )

            self.evaluate_and_log_model_if_improved(
                data=data,
                target="result",
                best_features=iteration_features,
                test_hyperparams=best_params,
                best_split_number=best_n_splits,
                best_splits=best_split,
                iteration_precision=iteration_precision,
                iteration_recall=iteration_recall,
                best_precision=self.precision_weighted_cv,
                best_recall=self.recall_weighted_cv
            )

            best_params, best_n_splits, best_split, optuna_precision, optuna_recall = self.run_optuna_precision(
                data, 
                iteration_features, 
                target,
                n_trials=1000,
                #forced_params=best_params,
                #forced_split=best_n_splits
            )

            self.evaluate_and_log_model_if_improved(
                data=data,
                target="result",
                best_features=iteration_features,
                test_hyperparams=best_params,
                best_split_number=best_n_splits,
                best_splits=best_split,
                iteration_precision=optuna_precision,
                iteration_recall=optuna_recall,
                best_precision=self.precision_weighted_cv,
                best_recall=self.recall_weighted_cv
            )

    def feature_importance(self):
        data = self._load_data()

        X = data[self.best_features]
        y = data[self.target]

        # Copiar los hiperparámetros optimizados y añadir forzadamente algunos necesarios
        params = self.best_params.copy()
        params.update({
            "random_state": self.random_state,
            "n_jobs": 1,
            "tree_method": "hist",
            "device": "cuda",
            "enable_categorical": True
        })

        final_model = XGBClassifier(**params)
        final_model.fit(X.fillna(0), y)

        # Obtener e imprimir importancia
        importances = final_model.feature_importances_
        feature_importance = pd.DataFrame({
            'Feature': self.best_features,
            'Importance': importances
        }).sort_values(by='Importance', ascending=False)

        # Plot de la importancia de las features
        plt.figure(figsize=(10, 6))
        plt.barh(feature_importance['Feature'], feature_importance['Importance'], color='skyblue')
        plt.xlabel("Feature Importance")
        plt.ylabel("Feature")
        plt.title("Feature Importance using Final Global Best XGBoost Model")
        plt.gca().invert_yaxis()
        plt.tight_layout()
        plt.show()

    def log_to_mlflow(self):
        self.number_experiment = 1 if self.number_experiment is None else self.number_experiment + 1
        run_name = f"XGBoost_{self.number_experiment}"
        description = self.description if self.description is not None else self.best_description

        experiment_name = "BetPredictions"
        tracking_uri = os.path.abspath("mlruns")
        mlflow.set_tracking_uri(f"file:///{tracking_uri}")
        mlflow.set_experiment(experiment_name)

        mlflow.sklearn.autolog(log_models=True, silent=True, log_input_examples=False, log_model_signatures=False)

        with mlflow.start_run(run_name=run_name, tags={"mlflow.note.content": description}):
            # ✅ Se asegura que se usen exactamente los mismos hiperparámetros
            model_params = self.best_params.copy()
            model_params.update({
                "random_state": self.random_state,
                "tree_method": "hist",
                "enable_categorical": True
            })

            self.best_model = XGBClassifier(**model_params)
            self.best_model.fit(self.X_final.fillna(0), self.y_final)

            mlflow.xgboost.log_model(self.best_model, artifact_path="model")

            # Log general metrics
            mlflow.log_metric("f1_weighted_cv", self.f1_weighted_cv)
            mlflow.log_metric("f1_macro_cv", self.f1_macro_cv)
            mlflow.log_metric("precision_weighted_cv", self.precision_weighted_cv)
            mlflow.log_metric("precision_macro_cv", self.precision_macro_cv)
            mlflow.log_metric("recall_weighted_cv", self.recall_weighted_cv)
            mlflow.log_metric("recall_macro_cv", self.recall_macro_cv)

            for param, value in self.best_params.items():
                mlflow.log_param(param, value)
            mlflow.log_param("Features", ",".join(self.best_features))
            mlflow.log_param("Best split number", int(self.best_split))

            mlflow.log_dict(self.cv_avg_class_metrics, "per_class_metrics.json")

            print(f"\nStored summary:")
            print(f"Weighted F1:       {self.f1_weighted_cv:.4f}")
            print(f"Macro F1:          {self.f1_macro_cv:.4f}")
            print(f"Weighted Precision:{self.precision_weighted_cv:.4f}")
            print(f"Macro Precision:   {self.precision_macro_cv:.4f}")
            print(f"Weighted Recall:   {self.recall_weighted_cv:.4f}")
            print(f"Macro Recall:      {self.recall_macro_cv:.4f}")

            print("\nCross-validation average per-class metrics:")
            for class_label, class_metrics in self.cv_avg_class_metrics.items():
                print(f"\nClass '{class_label}':")
                if not class_metrics:
                    print("  ⚠️ No metrics found for this class.")
                    continue
                for metric, value in class_metrics.items():
                    try:
                        print(f"  {metric}: {float(value):.4f}")
                    except (ValueError, TypeError):
                        print(f"  {metric}: {value} ⚠️ Invalid value")

            self.feature_importance()

            mlflow.sklearn.autolog(disable=True)
            mlflow.end_run()

    def process_feature_column(self, col_name: str, team: str, vs_team: str, current_date):
        """
        Analiza el nombre de una columna y llama directamente a la función de cálculo correspondiente,
        devolviendo el valor calculado (ya sea un promedio o una racha consecutiva).
        """

        stats = MatchStatsCalculator(self.seasons, self.long_strike, self.medium_strike, self.features_to_work, self.existing_csv)
        df = stats.get_dataframe()

        # Parsear el nombre
        result = {
            "type": None,
            "home_away": None,
            "column": None,
            "strike": None,
            "location": None,
            "vs_team": False,
            "between_mode": False,
            "opponent_mode": None,
            "operator": None,
            "value": None
        }

        if col_name.startswith("avg_"):
            result["type"] = "avg"
            match = re.match(r"avg_(home|away)_(.+?)_last_(\d+)_(.+)", col_name)
            if match:
                result["home_away"], result["column"], strike, suffix = match.groups()
                result["strike"] = int(strike)

                if "at_home" in suffix:
                    result["location"] = "home"
                elif "at_away" in suffix:
                    result["location"] = "away"
                else:
                    result["location"] = "both"

                if "_vs_" in suffix:
                    result["vs_team"] = True
                    result["between_mode"] = True
                elif "rivals_of_" in suffix:
                    result["vs_team"] = True
                    result["opponent_mode"] = "rival_avg" if "_at_" not in suffix else "rival_role_exact"

        elif col_name.startswith("consecutive_"):
            result["type"] = "consecutive"
            match = re.match(r"consecutive_(home|away)_(.+?)_(gt|lt|eq)_(\d+)(_.+)", col_name)
            if match:
                result["home_away"], result["column"], operator, value, suffix = match.groups()
                result["operator"] = operator
                result["value"] = int(value)

                if "at_home" in suffix:
                    result["location"] = "home"
                elif "at_away" in suffix:
                    result["location"] = "away"
                else:
                    result["location"] = "both"

                if "_vs_" in suffix:
                    result["vs_team"] = True
                    result["between_mode"] = True

        # Desempaquetar para claridad
        type_ = result["type"]
        home_away = result["home_away"]
        column = result["column"]
        strike = result["strike"]
        location = result["location"]
        between_mode = result["between_mode"]
        opponent_mode = result["opponent_mode"]
        operator = result["operator"]
        value = result["value"]

        # Lógica de ejecución
        if type_ == "avg":
            if result["vs_team"]:
                if between_mode:
                    return stats.calculate_avg_stat(
                        f"{home_away}_{column}", strike, team,
                        current_date, location=location, team2=vs_team, between_mode=True, df=df
                    )
                elif opponent_mode:
                    return stats.calculate_avg_stat(
                        f"{home_away}_{column}", strike, vs_team,
                        current_date, location=location, opponent_mode=opponent_mode, df=df
                    )
                else:
                    raise ValueError(f"Tipo de rival no reconocido en {col_name}")
            else:
                return stats.calculate_avg_stat(
                    f"{home_away}_{column}", strike, team,
                    current_date, location=location, df=df
                )

        elif type_ == "consecutive":
            return stats.calculate_consecutive_stat(
                f"{home_away}_{column}", value, team,
                current_date, location=location,
                team2=vs_team if result["vs_team"] else None,
                between_mode=between_mode,
                operator=operator, df=df
            )

        else:
            raise ValueError(f"Tipo desconocido en columna: {col_name}")

    def prepare_features(self):
        stats = MatchStatsCalculator(self.seasons, self.long_strike, self.medium_strike, self.features_to_work, self.existing_csv)
        data = stats.get_dataframe()

        self.fetch_latest_mlflow_run()

        match_date_str = input("Enter the match date (YYYY-MM-DD): ").strip()
        current_date = pd.Timestamp(match_date_str)

        num_matches = int(input("Please, tell me how many matches you want me to predict: "))
        match_data = []

        # Preguntar por la gameweek si es una feature relevante
        gameweek = None
        if 'gameweek' in self.best_features:
            gameweek_choice = input("Are the matches in the immediate next gameweek? (yes/no): ").strip().lower()
            if gameweek_choice in ["no", "n"]:
                gameweek = input("Enter the gameweek: ").strip()
            else:
                latest_match = data.sort_values(by="date", ascending=False).iloc[0]
                last_gameweek = latest_match.get("gameweek")
                if pd.notna(last_gameweek):
                    try:
                        last_gameweek = int(last_gameweek)
                        gameweek = last_gameweek + 1 if last_gameweek < 38 else 1
                    except ValueError:
                        print("Warning: last gameweek is not a valid integer.")
                        gameweek = None
                else:
                    print("Warning: last gameweek is NaN.")
                    gameweek = None

        for i in range(num_matches):
            input_features = {}

            for feature in self.best_features:
                if feature == 'gameweek':
                    input_features['gameweek'] = gameweek

                elif feature == 'home_team_name':
                    home_team_name = input("Enter the home team name: ").strip()
                    input_features['home_team_name'] = home_team_name

                elif feature == 'away_team_name':
                    away_team_name = input("Enter the away team name: ").strip()
                    input_features['away_team_name'] = away_team_name

                elif feature in ['hour_of_day', 'day_of_week', 'day_of_year']:
                    match_date = input("Enter the match date (YYYY-MM-DD): ").strip()
                    match_time = input("Enter the match time (HH:MM): ").strip()
                    match_date = pd.to_datetime(match_date, format="%Y-%m-%d", errors="coerce")
                    match_time = pd.to_datetime(match_time, format="%H:%M", errors="coerce")

                    if pd.isna(match_date) or pd.isna(match_time):
                        print("⚠️ Invalid date/time entered. Skipping feature generation.")
                        continue

                    input_features['day_of_week'] = match_date.dayofweek
                    input_features['day_of_year'] = match_date.dayofyear
                    input_features['hour_of_day'] = match_time.hour + match_time.minute / 60

                elif feature == 'referee':
                    referee = input("Enter the referee's name: ").strip()
                    input_features['referee'] = referee

                elif feature == 'var':
                    var = input("Enter the referee VAR's name: ").strip()
                    input_features['var'] = var

                elif feature == 'home_trainer':
                    # Igual que antes (coach home)
                    pass

                elif feature == 'away_trainer':
                    # Igual que antes (coach away)
                    pass

                elif feature == 'stadium':
                    # Igual que antes (stadium change)
                    pass

                elif feature == 'prob_home_avg':
                    cuota = input("Enter the HOME win ODDS from Bwin (not probability): ").strip()
                    cuota = float(cuota)
                    input_features['prob_home_avg'] = 1 / cuota  # convertir a probabilidad

                elif feature == 'prob_away_avg':
                    cuota = input("Enter the AWAY win ODDS from Bwin (not probability): ").strip()
                    cuota = float(cuota)
                    input_features['prob_away_avg'] = 1 / cuota

                elif feature == 'prob_draw_avg':
                    cuota = input("Enter the DRAW ODDS from Bwin (not probability): ").strip()
                    cuota = float(cuota)
                    input_features['prob_draw_avg'] = 1 / cuota

                elif feature in self.ranking_features:
                    # Si tienes lógica de ranking
                    pass

                else:
                    val = self.process_feature_column(feature, home_team_name, away_team_name, current_date)
                    input_features[feature] = val

            match_data.append(input_features.copy())

        # Convertir la lista de diccionarios a un DataFrame
        prepared_data = pd.DataFrame(match_data)

        # Aplicar LabelEncoder a todas las columnas categóricas
        for col in self.categorical_features:
            if col in prepared_data.columns and col in self.label_encoders:
                prepared_data[col] = prepared_data[col].map(
                    lambda x: self.label_encoders[col].transform([x])[0] if x in self.label_encoders[col].classes_ else -1
                )

        # Aplicar transformación circular a las columnas correspondientes
        for col, max_value in self.circular_max_values.items():
            if col in prepared_data.columns:
                prepared_data[f"{col}_sin"] = np.sin(2 * np.pi * prepared_data[col] / max_value)
                prepared_data[f"{col}_cos"] = np.cos(2 * np.pi * prepared_data[col] / max_value)
                prepared_data.drop(columns=[col], inplace=True)

        # Devolver el DataFrame preparado
        return prepared_data

    def predict(self):
        _ = self._load_data()
        
        match_data = self.prepare_features()

        # Copiar parámetros
        params = self.best_params.copy()

        # Asegurarse de tener un modelo reproducible
        params["random_state"] = self.random_state

        print("Loading XGBoost model...")
        model = xgb.XGBClassifier(**params)

        # Entrenar modelo con los datos históricos
        data = self._load_data()
        X = data[self.best_features]
        y = data[self.target]
        model.fit(X, y)

        # Convertir nuevos datos a DataFrame
        prepared_data = pd.DataFrame(match_data)

        # Asegurar que todas las columnas necesarias están presentes
        missing = set(self.best_features) - set(prepared_data.columns)
        if missing:
            raise ValueError(f"Missing features in prediction data: {missing}")

        # Ordenar columnas como en el entrenamiento
        prepared_data = prepared_data[self.best_features]

        print("Making predictions with trained XGBoost model...")

        probabilities = model.predict_proba(prepared_data)

        # Clases detectadas por el modelo (ej: [-1.0, 0.0, 1.0])
        class_order = model.classes_

        results = []
        for i, prob in enumerate(probabilities):
            prob_dict = dict(zip(class_order, prob))

            match_result = {
                "home_team_name": prepared_data.iloc[i]["home_team_name"],
                "away_team_name": prepared_data.iloc[i]["away_team_name"],
                "local_win_probability": prob_dict.get(2.0, 0.0) * 100,
                "draw_probability": prob_dict.get(1.0, 0.0) * 100,
                "away_win_probability": prob_dict.get(0.0, 0.0) * 100
            }
            results.append(match_result)

        return results

    def run(self):
        decision = input("What do you want to do? (1: Input more data, 2: Train the model, 3: Predict, 4: Open MLFlow): ").strip()
        if decision == "1":
            decision2 = input("Do you want to train the model after? (y/n): ").strip()

            if decision2 in ["yes", "y"]:
                calculator = MatchStatsCalculator(self.seasons, self.long_strike, self.medium_strike, self.features_to_work, self.existing_csv)
                calculator.update_or_add_columns()
                
                self.train()

            elif decision2 in ["no", "n"]:
                calculator = MatchStatsCalculator(self.seasons, self.long_strike, self.medium_strike, self.features_to_work, self.existing_csv)
                calculator.update_or_add_columns()

        elif decision == "2":
            self.train()

        elif decision == "3":
            predictions = self.predict()
            
            print("\n📊 Predictions for upcoming matches:")
            for i, prediction in enumerate(predictions, start=1):
                print(f"Prediction {i}: {prediction}")

        elif decision == "4":
            self.fetch_latest_mlflow_run()

# RUN FULL

In [9]:
seasons = [
    "2018-2019",
    "2019-2020",
    "2020-2021",
    "2021-2022",
    "2022-2023",
    "2023-2024",
    "2024-2025"
]

long_strike = 38
medium_strike = 19

categorical_features = ["home_team_name", "away_team_name", "home_trainer", "away_trainer", "stadium", "referee", "var"]

circular_max_values = {"hour_of_day": 24, "day_of_week": 7, "gameweek": 38, "day_of_year": 365}

ranking_features = ["home_team_rank", "away_team_rank", "home_team_points", "away_team_points", "home_team_goals_for", "away_team_goals_for", "home_team_goals_againsts",
            "away_team_goals_against", "home_team_goals_difference", "away_team_goals_difference", "prob_home_avg", "prob_draw_avg", "prob_away_avg"]

target = "result"

features_to_work = {
    "possession": [
        [20, 30, 40, 50, 60, 70, 80], 
        ["le", "le", "le", "ge", "ge", "ge", "ge"]],
    "goals": [
        [0, 1, 2, 3, 4, 5, 6], 
        ["eq", "ge", "ge", "ge", "ge", "ge", "ge"]],
    "PlayersAge": [
        [23.0, 27.0, 30.0, 30.0], 
        ["le", "le", "le", "gt"]],
    "PlayersShots": [
        [0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0], 
        ["eq", "eq", "eq", "eq", "eq", "eq", "eq", "eq", "eq", "eq", "eq", "eq", "eq", "eq", "eq", "ge"]],
    "PlayersShotsOnTarget": [
        [0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0], 
        ["eq", "eq", "eq", "eq", "eq", "eq", "eq", "eq", "eq", "eq", "ge"]],
    "PlayersCompletedPasses": [
        [100.0, 200.0, 300.0, 400.0, 500.0, 600.0], 
        ["le", "le", "le", "ge", "ge", "ge"]],
    "PlayersAttemptedPasses": [
        [200.0, 300.0, 400.0, 500.0, 600.0, 700.0, 800.0], 
        ["le", "le", "le", "le", "le", "le", "ge"]],
    "Players%CompletedPasses": [
        [40.0, 50.0, 60.0, 70.0, 80.0, 90.0], 
        ["le", "le", "ge", "ge", "ge", "ge"]],
    "PlayersDistancePasses": [
        [50, 100, 150, 200, 250, 300],
        ["le", "le", "le", "ge", "ge", "ge"]],
    "PlayersDistanceProgression": [
        [20, 40, 60, 80, 100, 120],
        ["le", "le", "le", "ge", "ge", "ge"]],
    "PlayersShortPasses": [
        [50, 100, 150, 200, 250],
        ["le", "le", "le", "ge", "ge"]],
    "PlayersAttemptedShortPasses": [
        [60, 120, 180, 240, 300],
        ["le", "le", "le", "ge", "ge"]],
    "Players%ShortCompletedPasses": [
        [60, 70, 80, 90],
        ["le", "le", "ge", "ge"]],
    "PlayersMediumPasses": [
        [30, 60, 90, 120, 150],
        ["le", "le", "le", "ge", "ge"]],
    "PlayersAttemptedMediumPasses": [
        [40, 80, 120, 160, 200],
        ["le", "le", "le", "ge", "ge"]],
    "Players%MediumCompletedPasses": [
        [60, 70, 80, 90],
        ["le", "le", "ge", "ge"]],
    "PlayersLongPasses": [
        [10, 20, 30, 40, 50],
        ["le", "le", "le", "ge", "ge"]],
    "PlayersAttemptedLongPasses": [
        [15, 30, 45, 60, 75],
        ["le", "le", "le", "ge", "ge"]],
    "Players%LongCompletedPasses": [
        [40, 50, 60, 70],
        ["le", "le", "ge", "ge"]],
    "PlayersAssistance": [
        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],  
        ["eq", "ge", "ge", "ge", "ge", "ge", "ge", "ge", "ge", "ge", "ge"]],
    "PlayersExpectedGoalsAssistance": [
        [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],  
        ["le", "le", "le", "le", "ge", "ge", "ge", "ge", "ge", "ge"]],
    "PlayersExpectedAssistance": [
        [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],  
        ["le", "le", "le", "le", "ge", "ge", "ge", "ge", "ge", "ge"]],
    "PlayersKeyPasses": [
        [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],  
        ["eq", "ge", "ge", "ge", "ge", "ge", "ge", "ge", "ge", "ge"]],
    "PlayersLast1/3Passes": [
        [5, 10, 15, 20, 25, 30, 35, 40, 45, 50],  
        ["le", "le", "le", "le", "le", "ge", "ge", "ge", "ge", "ge"]],
    "PlayersGoalAreaPasses": [
        [1, 3, 5, 7, 9, 11, 13, 15, 17, 19],  
        ["le", "le", "ge", "ge", "ge", "ge", "ge", "ge", "ge", "ge"]],
    "PlayersGoalAreaCrosses": [
        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],  
        ["eq", "ge", "ge", "ge", "ge", "ge", "ge", "ge", "ge", "ge", "ge"]],
    "PlayersGoalPasses": [
        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],  
        ["eq", "ge", "ge", "ge", "ge", "ge", "ge", "ge", "ge", "ge", "ge"]],
    "PlayersLiveBallPasses": [
        [20, 40, 60, 80, 100, 120, 140, 160, 180, 200],  
        ["le", "le", "le", "le", "le", "ge", "ge", "ge", "ge", "ge"]],
    "PlayersDeadBallPasses": [
        [5, 10, 15, 20, 25, 30, 35, 40, 45, 50],  
        ["le", "le", "le", "le", "le", "ge", "ge", "ge", "ge", "ge"]],
    "PlayersFreeKick": [
        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],  
        ["eq", "ge", "ge", "ge", "ge", "ge", "ge", "ge", "ge", "ge", "ge"]],
    "PlayersSidePasses": [
        [10, 20, 30, 40, 50, 60, 70, 80, 90, 100],  
        ["le", "le", "le", "le", "le", "ge", "ge", "ge", "ge", "ge"]],
    "result": [
        [-1, 0, 1, -1, 1], 
        ["eq", "eq", "eq", "gt", "lt"]],
    "PlayersCrosses": [
        [5, 10, 15, 20, 25, 30, 35, 40, 45, 50],  
        ["le", "le", "le", "le", "le", "ge", "ge", "ge", "ge", "ge"]],
    "PlayersStrongcrosses": [
        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],  
        ["eq", "ge", "ge", "ge", "ge", "ge", "ge", "ge", "ge", "ge", "ge"]],
    "PlayersCorner": [
        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],  
        ["eq", "ge", "ge", "ge", "ge", "ge", "ge", "ge", "ge", "ge", "ge"]],
    "PlayersCornerIn": [
        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],  
        ["eq", "ge", "ge", "ge", "ge", "ge", "ge", "ge", "ge", "ge", "ge"]],
    "PlayersCornerOut": [
        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],  
        ["eq", "ge", "ge", "ge", "ge", "ge", "ge", "ge", "ge", "ge", "ge"]],
    "PlayersCornerRect": [
        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],  
        ["eq", "ge", "ge", "ge", "ge", "ge", "ge", "ge", "ge", "ge", "ge"]],
    "PlayersOffsidePasses": [
        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],  
        ["eq", "ge", "ge", "ge", "ge", "ge", "ge", "ge", "ge", "ge", "ge"]],
    "PlayersPassesBlocked": [
        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],  
        ["eq", "ge", "ge", "ge", "ge", "ge", "ge", "ge", "ge", "ge", "ge"]],
    "PlayersTackles": [
        [5, 10, 15, 20, 25, 30, 35, 40, 45, 50],  
        ["le", "le", "le", "le", "le", "ge", "ge", "ge", "ge", "ge"]],
    "PlayersSuccessfulTackles": [
        [0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20],  
        ["eq", "ge", "ge", "ge", "ge", "ge", "ge", "ge", "ge", "ge", "ge"]],
    "PlayersTacklesInDefense": [
        [0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20],  
        ["eq", "ge", "ge", "ge", "ge", "ge", "ge", "ge", "ge", "ge", "ge"]],
    "PlayersTacklesInMedium": [
        [0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20],  
        ["eq", "ge", "ge", "ge", "ge", "ge", "ge", "ge", "ge", "ge", "ge"]],
    "PlayersTacklesInAttack": [
        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],  
        ["eq", "ge", "ge", "ge", "ge", "ge", "ge", "ge", "ge", "ge", "ge"]],
    "PlayersDribblerTackles": [
        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],  
        ["eq", "ge", "ge", "ge", "ge", "ge", "ge", "ge", "ge", "ge", "ge"]],
    "PlayersAttemptedDribblerTackles": [
        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],  
        ["eq", "ge", "ge", "ge", "ge", "ge", "ge", "ge", "ge", "ge", "ge"]],
    "Players%DribblerTacklesCompleted": [
        [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100],  
        ["le", "le", "le", "le", "le", "ge", "ge", "ge", "ge", "ge"]],
    "PlayersDribblerTacklesNonCompleted": [
        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],  
        ["eq", "ge", "ge", "ge", "ge", "ge", "ge", "ge", "ge", "ge", "ge"]],
    "PlayersBallsBlocked": [
        [0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20],  
        ["eq", "ge", "ge", "ge", "ge", "ge", "ge", "ge", "ge", "ge", "ge"]],
    "PlayersShotsBlocked": [
        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],  
        ["eq", "ge", "ge", "ge", "ge", "ge", "ge", "ge", "ge", "ge", "ge"]],
    "PlayersInterceptions": [
        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],  
        ["eq", "ge", "ge", "ge", "ge", "ge", "ge", "ge", "ge", "ge", "ge"]],
    "PlayersTackles+Interceptions": [
        [0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20],  
        ["eq", "ge", "ge", "ge", "ge", "ge", "ge", "ge", "ge", "ge", "ge"]],
    "PlayersClearances": [
        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],  
        ["eq", "ge", "ge", "ge", "ge", "ge", "ge", "ge", "ge", "ge", "ge"]],
    "PlayersMistakesRivalShots": [
        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],  
        ["eq", "ge", "ge", "ge", "ge", "ge", "ge", "ge", "ge", "ge", "ge"]],
    "PlayersTouches": [
        [20, 30, 40, 50, 60, 70, 80, 90, 100],  
        ["le", "le", "le", "le", "le", "ge", "ge", "ge", "ge", "ge"]],
    "PlayersOwnPenaltyAreaTouches": [
        [0, 2, 4, 6, 8, 10],  
        ["eq", "ge", "ge", "ge", "ge", "ge"]],
    "PlayersTouchesInDefense": [
        [0, 5, 10, 15, 20, 25],  
        ["le", "le", "le", "le", "le", "ge", "ge", "ge", "ge", "ge"]],
    "PlayersTouchesInMedium": [
        [0, 5, 10, 15, 20, 25],  
        ["le", "le", "le", "le", "le", "ge", "ge", "ge", "ge", "ge"]],
    "PlayersTouchesInAttack": [
        [0, 5, 10, 15, 20, 25],  
        ["le", "le", "le", "le", "le", "ge", "ge", "ge", "ge", "ge"]],
    "PlayersAwayPenaltyAreaTouches": [
        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],  
        ["eq", "ge", "ge", "ge", "ge", "ge", "ge", "ge", "ge", "ge", "ge"]],
    "PlayersLiveBallTouches": [
        [10, 20, 30, 40, 50, 60, 70, 80, 90, 100],  
        ["le", "le", "le", "le", "le", "ge", "ge", "ge", "ge", "ge"]],
    "PlayersAttemptedDribbles": [
        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],  
        ["eq", "ge", "ge", "ge", "ge", "ge", "ge", "ge", "ge", "ge"]],
    "PlayersDribblesCompleted": [
        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],  
        ["eq", "ge", "ge", "ge", "ge", "ge", "ge", "ge", "ge", "ge"]],
    "Players%DribblesCompleted": [
        [0, 20, 40, 60, 80, 100],  
        ["le", "le", "le", "le", "le", "ge", "ge", "ge", "ge", "ge"]],
    "PlayersBallCarries": [
        [0, 10, 20, 30, 40, 50, 60, 70],  
        ["eq", "ge", "ge", "ge", "ge", "ge", "ge", "ge", "ge"]], 
    "PlayersDistanceCarried": [
        [0, 10, 20, 30, 40, 50, 60],  
        ["eq", "ge", "ge", "ge", "ge", "ge", "ge"]],
    "PlayersForwardDistanceCarried": [
        [0, 5, 10, 15, 20, 25],  
        ["eq", "ge", "ge", "ge", "ge", "ge"]],
    "PlayersForwardCarries": [
        [0, 1, 2, 3, 4, 5, 6, 7],  
        ["eq", "ge", "ge", "ge", "ge", "ge"]],
    "PlayersCarriesInAttack": [
        [0, 1, 2, 3, 4, 5, 6],  
        ["eq", "ge", "ge", "ge", "ge", "ge"]],
    "PlayersAwayPenaltyAreaCarries": [
        [0, 1, 2, 3, 4, 5, 6],  
        ["eq", "ge", "ge", "ge", "ge", "ge"]],
    "PlayersLostControlCarries": [
        [0, 1, 2, 3, 4, 5],  
        ["eq", "ge", "ge", "ge", "ge", "ge"]],
    "PlayersLostCarries": [
        [0, 1, 2, 3, 4, 5],  
        ["eq", "ge", "ge", "ge", "ge", "ge"]],
    "PlayersPassesReception": [
        [0, 10, 20, 30, 40, 50, 60, 70],  
        ["eq", "ge", "ge", "ge", "ge", "ge", "ge"]],
    "PlayersAttackPassesReception": [
        [0, 5, 10, 15, 20],  
        ["eq", "ge", "ge", "ge", "ge"]],
    "PlayersYellowCards": [
        [0, 1, 2, 3, 4],  
        ["eq", "ge", "ge", "ge", "ge"]],
    "PlayersRedCards": [
        [0, 1, 2, 3],  
        ["eq", "ge", "ge", "ge"]],
    "PlayersSecondYellowCards": [
        [0, 1, 2],  
        ["eq", "ge", "ge"]],
    "PlayersFouls": [
        [0, 10, 20, 30, 40, 50, 60, 70, 80, 90],  
        ["eq", "ge", "ge", "ge", "ge", "ge", "ge", "ge"]],
    "PlayersFoulsReceived": [
        [0, 10, 20, 30, 40, 50, 60, 70, 80, 90],  
        ["eq", "ge", "ge", "ge", "ge", "ge", "ge", "ge"]],
    "PlayersPenalties": [
        [0, 1, 2, 3],  
        ["eq", "ge", "ge", "ge"]],
    "PlayersPenaltiesConceded": [
        [0, 1, 2, 3],  
        ["eq", "ge", "ge", "ge"]],
    "PlayersLostBallRecoveries": [
        [0, 10, 20, 30, 40, 50],  
        ["eq", "ge", "ge", "ge", "ge"]],
    "PlayersAerialsWon": [
        [0, 10, 20, 30, 40, 50, 60, 70],  
        ["eq", "ge", "ge", "ge", "ge", "ge"]],
    "PlayersAerialsLost": [
        [0, 10, 20, 30, 40, 50],  
        ["eq", "ge", "ge", "ge", "ge"]],
    "Players%AerialsWon": [
        [0, 10, 20, 30, 40, 50, 60],  
        ["eq", "ge", "ge", "ge", "ge"]],
    "KeepersShotsOnTargetAgainst": [
        [0, 1, 2, 3, 4, 5, 6],
        ["eq", "ge", "ge", "ge", "ge", "ge"]],
    "KeepersGoalsAgainst": [
        [0, 1, 2, 3, 4, 5],
        ["eq", "ge", "ge", "ge", "ge"]],
    "KeepersSaved": [
        [0, 1, 2, 3, 4, 5],
        ["eq", "ge", "ge", "ge", "ge"]],
    "Keepers%Saved": [
        [0, 10, 20, 30, 40, 50, 60],
        ["eq", "ge", "ge", "ge", "ge"]],
    "KeepersxG": [
        [0, 0.1, 0.2, 0.3, 0.4, 0.5],
        ["eq", "ge", "ge", "ge", "ge"]],
    "KeepersPassesLaunched": [
        [0, 1, 2, 3, 4, 5],
        ["eq", "ge", "ge", "ge", "ge"]],
    "KeepersAttemptedPassesLaunched": [
        [0, 1, 2, 3, 4, 5],
        ["eq", "ge", "ge", "ge", "ge"]],
    "Keepers%CompletedPassesLaunched": [
        [0, 20, 40, 60, 80, 100],
        ["eq", "ge", "ge", "ge", "ge"]],
    "KeepersPasses": [
        [0, 1, 2, 3, 4, 5],
        ["eq", "ge", "ge", "ge", "ge"]],
    "KeepersAttemptedPasses": [
        [0, 1, 2, 3, 4, 5],
        ["eq", "ge", "ge", "ge", "ge"]],
    "Keepers%CompletedPasses": [
        [0, 20, 40, 60, 80, 100],
        ["eq", "ge", "ge", "ge", "ge"]],
    "KeepersPassesDistance": [
        [0, 10, 20, 30, 40, 50],
        ["eq", "ge", "ge", "ge", "ge"]],
    "KeepersAttemptedKicks": [
        [0, 1, 2, 3, 4],
        ["eq", "ge", "ge", "ge", "ge"]],
    "Keepers%Kicks": [
        [0, 20, 40, 60, 80, 100],
        ["eq", "ge", "ge", "ge", "ge"]],
    "KeepersKicksDistance": [
        [0, 10, 20, 30, 40, 50],
        ["eq", "ge", "ge", "ge", "ge"]],
    "KeepersCrosses": [
        [0, 1, 2, 3, 4],
        ["eq", "ge", "ge", "ge", "ge"]],
    "KeepersCrossesStopped": [
        [0, 1, 2, 3, 4],
        ["eq", "ge", "ge", "ge", "ge"]],
    "Keepers%CrossesStopped": [
        [0, 20, 40, 60, 80, 100],
        ["eq", "ge", "ge", "ge", "ge"]],
    "KeepersActionsOutsideArea": [
        [0, 1, 2, 3, 4],
        ["eq", "ge", "ge", "ge", "ge"]],
    "KeepersDistanceActionsArea": [
        [0, 10, 20, 30, 40, 50],
        ["eq", "ge", "ge", "ge", "ge"]]
}

existing_csv = "dataset_2.csv"

random_state = 42

pipeline = ModelPipeline(existing_csv, categorical_features, circular_max_values, ranking_features, target, random_state, seasons, long_strike, medium_strike, features_to_work)

pipeline.run()

Se está abriendo en una sesión de navegador existente.
